<a href="https://colab.research.google.com/github/codehey/FInalADM2023/blob/ADM/Bias_in_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np
import re
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# Clean text data
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [155]:
cd /content/drive/MyDrive/Datasets


/content/drive/MyDrive/Datasets


In [156]:
!git init

Initialized empty Git repository in /content/drive/MyDrive/Datasets/.git/


In [157]:
!git add .


In [158]:
!git commit -m "FinalADM2023"



*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@d304f8c6b98b.(none)')


In [169]:
!git config --global user.email "alnu8@hawk.iit.edu"

In [170]:
!git config --global user.name "codehey"

In [172]:
!git remote add origin https://github.com/codehey/FInalADM2023

fatal: remote origin already exists.


In [168]:
!git status

On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   Equity-Evaluation-Corpus.csv
	new file:   data_processed_test.csv
	new file:   data_processed_train.csv
	new file:   data_processed_validate.csv
	new file:   svm-in-twitter-dataset-449ca3a3aa0d.json



In [174]:
!git branch

In [176]:
!git push -u origin ADM

error: src refspec ADM does not match any
error: failed to push some refs to 'https://github.com/codehey/FInalADM2023'


In [173]:
!git push -u origin master

error: src refspec master does not match any
error: failed to push some refs to 'https://github.com/codehey/FInalADM2023'


In [31]:
train_data = pd.read_csv("/content/drive/My Drive/Datasets/data_processed_train.csv")
test_data = pd.read_csv("/content/drive/My Drive/Datasets/data_processed_test.csv")
val_data = pd.read_csv("/content/drive/My Drive/Datasets/data_processed_validate.csv")

In [32]:
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # # Remove urls
    # text = re.sub(r'http\S+', '', text)
    # # Remove mentions
    # text = re.sub(r'@\S+', '', text)
    # # Remove numbers
    # text = re.sub(r'\d+', '', text)
    # # Remove punctuation
    # text = re.sub(r'[^\w\s]', '', text)
    # # Remove stopwords and lemmatize
    # # text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

train_data['text'] = train_data['text'].apply(clean_text)
test_data['text'] = test_data['text'].apply(clean_text)
val_data['text'] = val_data['text'].apply(clean_text)

In [33]:
train_data.head()
test_data.head()
val_data.head()

,target,text
0,-1,fuck creep need someone protect me
1,-1,jeesh go sjbc ride cant there
2,-1,rehearse today ny again kinda cold crave pancake
3,1,cant wait 2mz worra lush half term fairplayys xx
4,1,never happenunless plane go


In [34]:
print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(10000, 2)
(5000, 2)
(2000, 2)


In [35]:
print(train_data.isnull().sum())
print(test_data.isnull().sum())
print(val_data.isnull().sum())

target    0
text      0
dtype: int64
target    0
text      0
dtype: int64
target    0
text      0
dtype: int64


In [36]:
train_data = train_data.dropna()
test_data = test_data.dropna()
val_data = val_data.dropna()

In [37]:
print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(10000, 2)
(5000, 2)
(2000, 2)


In [38]:
# Balancing train data
counts = train_data['target'].value_counts()
min_count = counts.min()

train_data = pd.concat([train_data[train_data['target'] == t].sample(min_count) for t in counts.index])

In [39]:
# balancing validation dataset
counts = val_data['target'].value_counts()
min_count = counts.min()

val_data = pd.concat([val_data[val_data['target'] == t].sample(min_count) for t in counts.index])

In [40]:
# balancing Test dataset
counts = test_data['target'].value_counts()
min_count = counts.min()

test_data = pd.concat([test_data[test_data['target'] == t].sample(min_count) for t in counts.index])

In [41]:
print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(9972, 2)
(4958, 2)
(1976, 2)


In [42]:
print(train_data['target'].value_counts())
print(test_data['target'].value_counts())
print(val_data['target'].value_counts())

-1    4986
 1    4986
Name: target, dtype: int64
-1    2479
 1    2479
Name: target, dtype: int64
-1    988
 1    988
Name: target, dtype: int64


Predictions Using CountVectorizer

In [43]:
from sklearn.svm import SVC
import time
start_time = time.time()
# Create document term matrix using CountVectorizer
count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(train_data['text'])

X_test_count = count_vectorizer.transform(test_data['text'])

# Define the pipeline for CountVectorizer
pipeline_count = Pipeline([
    ('clf', SVC(kernel='linear'))
    
])

# Fit the model using CountVectorizer
pipeline_count.fit(X_train_count, train_data['target'])
end_time = time.time()
print("Time taken to train the model: ", end_time - start_time, " seconds")

start_time = time.time()
y_pred_count = pipeline_count.predict(X_test_count)
end_time = time.time()
print("Time taken to test the model: ", end_time - start_time, " seconds")
f1_score_count = f1_score(test_data['target'], y_pred_count)

# Print the confusion matrix with labels
cm = confusion_matrix(test_data['target'], y_pred_count)
print("Confusion Matrix:")
print(cm)

# Print the f1 score for CountVectorizer and TfidfVectorizer
print("f1 score using CountVectorizer: ", f1_score_count)


Time taken to train the model:  16.194074153900146  seconds
Time taken to test the model:  2.533802032470703  seconds
Confusion Matrix:
[[1752  727]
 [ 727 1752]]
f1 score using CountVectorizer:  0.7067365873336022


Predictions Using TfidfVectorizer

In [44]:
import time
from sklearn.svm import SVC

start_time = time.time()
# Create document term matrix using TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['text'])

# Define the pipeline for TfidfVectorizer
pipeline_tfidf = Pipeline([
    ('clf', SVC())
])

# Fit the model using TfidfVectorizer
pipeline_tfidf.fit(X_train_tfidf, train_data['target'])
end_time = time.time()
print("Time taken to train the model: ", end_time - start_time, " seconds")

start_time = time.time()
X_test_tfidf = tfidf_vectorizer.transform(test_data['text'])
y_pred_tfidf = pipeline_tfidf.predict(X_test_tfidf)
end_time = time.time()
print("Time taken to test the model: ", end_time - start_time, " seconds")

f1_score_tfidf = f1_score(test_data['target'], y_pred_tfidf)

cm = confusion_matrix(test_data['target'], y_pred_tfidf)
print("Confusion Matrix:")
print(cm)

print("f1 score using TfidfVectorizer: ", f1_score_tfidf)


Time taken to train the model:  11.401472330093384  seconds
Time taken to test the model:  3.837346076965332  seconds
Confusion Matrix:
[[1808  671]
 [ 646 1833]]
f1 score using TfidfVectorizer:  0.7357013847080073


**Hyper parameter tunning for Countvectorizer**

In [45]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, f1_score,accuracy_score

# Create document term matrix using CountVectorizer
count_vectorizer = CountVectorizer(ngram_range=(1,1))

# Define the pipeline for CountVectorizer with hyperparameters to tune
pipeline_count = Pipeline([
    ('vect', count_vectorizer),
    ('clf', SVC(kernel='linear', C=1.0, max_iter=1000, probability=True))
])

# Define hyperparameters to tune for CountVectorizer
param_grid_count = {    
    'clf__C': [ 1],
    'clf__kernel': ['linear'],
    'clf__degree': [2],
    'clf__gamma': [ 0.1],
}

# Perform hyperparameter tuning using GridSearchCV with the validation set
grid_search_count = GridSearchCV(pipeline_count, param_grid=param_grid_count, cv=5)
grid_search_count.fit(val_data['text'], val_data['target'])

for i in range(len(grid_search_count.cv_results_['params'])):
    print(grid_search_count.cv_results_['params'][i], " -> ", grid_search_count.cv_results_['mean_test_score'][i])

# Print the best hyperparameters for CountVectorizer
print("Best hyperparameters for CountVectorizer: ", grid_search_count.best_params_)

# Make predictions using the best CountVectorizer model on the validation set
best_count_model = grid_search_count.best_estimator_
print(best_count_model)

y_pred = best_count_model.predict(val_data['text'])

accuracy = accuracy_score(val_data['target'], y_pred)

# Print the accuracy of the best CountVectorizer model on the validation set
print("Accuracy of the best CountVectorizer model on the validation set: ", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider

{'clf__C': 1, 'clf__degree': 2, 'clf__gamma': 0.1, 'clf__kernel': 'linear'}  ->  0.658401738908068
Best hyperparameters for CountVectorizer:  {'clf__C': 1, 'clf__degree': 2, 'clf__gamma': 0.1, 'clf__kernel': 'linear'}
Pipeline(steps=[('vect', CountVectorizer()),
                ('clf',
                 SVC(C=1, degree=2, gamma=0.1, kernel='linear', max_iter=1000,
                     probability=True))])
Accuracy of the best CountVectorizer model on the validation set:  0.979251012145749


In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score,classification_report



# Drop any rows that contain NaN values
train_data.dropna(inplace=True)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_data['text'], train_data['target'], test_size=0.2, random_state=42)

# Convert the preprocessed text data into feature vectors using TF-IDF vectorization
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Train an SVM model on the training set
model = svm.SVC(kernel='linear')
model.fit(X_train, y_train)

# Evaluate the model on the testing set and print the results

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

report = classification_report(y_test, y_pred, digits=4)
print('Classification report:')
print(report)

Accuracy: 0.7203007518796992
Classification report:
              precision    recall  f1-score   support

          -1     0.7345    0.7085    0.7213      1019
           1     0.7065    0.7326    0.7193       976

    accuracy                         0.7203      1995
   macro avg     0.7205    0.7206    0.7203      1995
weighted avg     0.7208    0.7203    0.7203      1995



Best Hyperparameter:-----



In [47]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import numpy as np
import time

# Define hyperparameters to tune for SVM
param_grid_svm = {
    'clf__C': [5],
    'clf__kernel': ['linear','rbf'],
    'clf__degree': [2],
    'clf__gamma': [ 0.1],
}

# Create document term matrix using CountVectorizer
count_vectorizer = CountVectorizer(ngram_range=(1,1))

# Define the pipeline for SVM
pipeline_svm = Pipeline([
    ('vect', count_vectorizer),
    ('clf', SVC(kernel='linear', C=1.0, max_iter=1000, probability=True))
])

# Perform hyperparameter tuning using GridSearchCV with the validation set
grid_search_svm = GridSearchCV(pipeline_svm, param_grid=param_grid_svm, cv=3)
grid_search_svm.fit(train_data['text'], train_data['target'])

# Print the best hyperparameters for SVM
print("Best hyperparameters for SVM: ", grid_search_svm.best_params_)

# Fit the model using SVM
start_time = time.time()
svm_model = SVC(C=grid_search_svm.best_params_['clf__C'], kernel=grid_search_svm.best_params_['clf__kernel'], gamma=grid_search_svm.best_params_['clf__gamma'])
svm_model.fit(X_train_count, train_data['target'])
end_time = time.time()
print("Time taken to train the model: ", end_time - start_time, " seconds")

# Fit the CountVectorizer on the training data
X_train_count = count_vectorizer.fit_transform(train_data['text'])

# Make predictions using the best SVM model on the test set
start_time = time.time()
X_test_count = count_vectorizer.transform(test_data['text'])
y_pred_svm = svm_model.predict(X_test_count)
end_time = time.time()
print("Time taken to test the model: ", end_time - start_time, " seconds")

# Generate classification report for SVM
report_svm = classification_report(test_data['target'], y_pred_svm)
print("Classification report using SVM:\n", report_svm)

# Print the accuracy of the best SVM model on the test set
accuracy_svm = accuracy_score(test_data['target'], y_pred_svm)
print("Accuracy of the best SVM model on the test set: ", accuracy_svm)

# Print the f1 score for SVM
f1_score_svm = f1_score(test_data['target'], y_pred_svm)
print("f1 score using SVM: ", f1_score_svm)

# Print the confusion matrix for SVM
cm_svm = confusion_matrix(test_data['target'], y_pred_svm)
print("Confusion Matrix:\n", cm_svm)

# Print the precision and recall for SVM
precision_svm = precision_score(test_data['target'], y_pred_svm)
recall_svm = recall_score(test_data['target'], y_pred_svm)




/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider

Best hyperparameters for SVM:  {'clf__C': 5, 'clf__degree': 2, 'clf__gamma': 0.1, 'clf__kernel': 'rbf'}
Time taken to train the model:  24.133077144622803  seconds
Time taken to test the model:  3.675107955932617  seconds
Classification report using SVM:
               precision    recall  f1-score   support

          -1       0.73      0.72      0.73      2479
           1       0.72      0.74      0.73      2479

    accuracy                           0.73      4958
   macro avg       0.73      0.73      0.73      4958
weighted avg       0.73      0.73      0.73      4958

Accuracy of the best SVM model on the test set:  0.727914481645825
f1 score using SVM:  0.7299299299299299
Confusion Matrix:
 [[1786  693]
 [ 656 1823]]


TfiDf Vectorizer with best hyperparameters

In [48]:
start_time = time.time()
# Create document term matrix using TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1))
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['text'])

# Define hyperparameters to tune for SVM
param_grid_svm = {
    'clf__C': [1],
    'clf__kernel': ['linear','rbf'],
    'clf__degree': [2],
    'clf__gamma': ['scale'],
}

# Define the pipeline for SVM with TfidfVectorizer
pipeline_tfidf_svm_best = Pipeline([
    ('vect', tfidf_vectorizer),
    ('clf', SVC())
])

# Perform hyperparameter tuning using GridSearchCV with the validation set
grid_search_svm_tfidf = GridSearchCV(pipeline_tfidf_svm_best, param_grid=param_grid_svm, cv=3)
grid_search_svm_tfidf.fit(train_data['text'], train_data['target'])

# Print the best hyperparameters for SVM with TfidfVectorizer
print("Best hyperparameters for SVM with TfidfVectorizer: ", grid_search_svm_tfidf.best_params_)

# Fit the model using SVM with TfidfVectorizer
svm_model_tfidf = SVC(C=grid_search_svm_tfidf.best_params_['clf__C'], kernel=grid_search_svm_tfidf.best_params_['clf__kernel'], gamma=grid_search_svm_tfidf.best_params_['clf__gamma'])
svm_model_tfidf.fit(X_train_tfidf, train_data['target'])
end_time = time.time()
print("Time taken to train the model: ", end_time - start_time, " seconds")

start_time = time.time()
X_test_tfidf = tfidf_vectorizer.transform(test_data['text'])
# Predict using the test data
y_pred_svm_tfidf = svm_model_tfidf.predict(X_test_tfidf)
end_time = time.time()
print("Time taken to test the model: ", end_time - start_time, " seconds")

# Generate classification report for SVM with TfidfVectorizer
report_svm_tfidf = classification_report(test_data['target'], y_pred_svm_tfidf)
print("Classification report using SVM with TfidfVectorizer:\n", report_svm_tfidf)

# Print the accuracy of the best SVM model on the test set
accuracy_svm_tfidf = accuracy_score(test_data['target'], y_pred_svm_tfidf)
print("Accuracy of the best SVM model with TfidfVectorizer on the test set: ", accuracy_svm_tfidf)

# Print the f1 score for SVM with TfidfVectorizer
f1_score_svm_tfidf = f1_score(test_data['target'], y_pred_svm_tfidf)
print("f1 score using SVM with TfidfVectorizer: ", f1_score_svm_tfidf)

# Print the confusion matrix for SVM with TfidfVectorizer
cm_svm_tfidf = confusion_matrix(test_data['target'], y_pred_svm_tfidf)
print("Confusion Matrix:\n", cm_svm_tfidf)

# Print the precision and recall for SVM with TfidfVectorizer
precision_svm_tfidf = precision_score(test_data['target'], y_pred_svm_tfidf)
recall_svm_tfidf = recall_score(test_data['target'], y_pred_svm_tfidf)
print("Precision using SVM with TfidfVectorizer: ", precision_svm_tfidf)
print("Recall using SVM with TfidfVectorizer: ", recall_svm_tfidf)


Best hyperparameters for SVM with TfidfVectorizer:  {'clf__C': 1, 'clf__degree': 2, 'clf__gamma': 'scale', 'clf__kernel': 'rbf'}
Time taken to train the model:  59.75414490699768  seconds
Time taken to test the model:  4.880526781082153  seconds
Classification report using SVM with TfidfVectorizer:
               precision    recall  f1-score   support

          -1       0.74      0.73      0.73      2479
           1       0.73      0.74      0.74      2479

    accuracy                           0.73      4958
   macro avg       0.73      0.73      0.73      4958
weighted avg       0.73      0.73      0.73      4958

Accuracy of the best SVM model with TfidfVectorizer on the test set:  0.7343686970552642
f1 score using SVM with TfidfVectorizer:  0.7357013847080073
Confusion Matrix:
 [[1808  671]
 [ 646 1833]]
Precision using SVM with TfidfVectorizer:  0.7320287539936102
Recall using SVM with TfidfVectorizer:  0.7394110528438886


In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# Create document term matrix using TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1))
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['text'])
X_test_tfidf = tfidf_vectorizer.transform(test_data['text'])

# Define hyperparameters to tune for SVM
param_grid_svm = {
    'clf__C': [1],
    'clf__kernel': ['linear','rbf'],
    'clf__degree': [2],
    'clf__gamma': ['scale'],
}


# Define the pipeline for TfidfVectorizer with SVM classifier
pipeline_tfidf_svm_best = Pipeline([
    ('clf', SVC(kernel='linear', C=1.0, max_iter=1000, probability=True))
])

# Fit the model using TfidfVectorizer with SVM classifier
pipeline_tfidf_svm_best.fit(X_train_tfidf, train_data['target'])

# Predict using the test data
y_pred_tfidf = pipeline_tfidf_svm_best.predict(X_test_tfidf)

# Generate classification report for TfidfVectorizer with SVM classifier
report_tfidf_svm = classification_report(test_data['target'], y_pred_tfidf)
print("Classification report using TfidfVectorizer with SVM classifier:\n", report_tfidf_svm)

# Generate confusion matrix for TfidfVectorizer with SVM classifier
cm = confusion_matrix(test_data['target'], y_pred_tfidf)
print("Confusion Matrix:")
print(cm)

# Calculate f1 score
f1_score_tfidf_svm = f1_score(test_data['target'], y_pred_tfidf)

# Print the f1 score
print("f1 score using TfidfVectorizer with SVM classifier: ", f1_score_tfidf_svm)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Classification report using TfidfVectorizer with SVM classifier:
               precision    recall  f1-score   support

          -1       0.65      0.61      0.63      2479
           1       0.63      0.68      0.65      2479

    accuracy                           0.64      4958
   macro avg       0.64      0.64      0.64      4958
weighted avg       0.64      0.64      0.64      4958

Confusion Matrix:
[[1511  968]
 [ 801 1678]]
f1 score using TfidfVectorizer with SVM classifier:  0.6548292682926828


**Measuring Bias**

In [50]:
data=pd.read_csv("/content/drive/My Drive/Datasets/Equity-Evaluation-Corpus.csv")

In [51]:
data['Sentence'].nunique()

8640

In [52]:
data.isnull().sum()

ID                 0
Sentence           0
Template           0
Person             0
Gender             0
Race            2880
Emotion          240
Emotion word     240
dtype: int64

In [53]:
data.columns

Index(['ID', 'Sentence', 'Template', 'Person', 'Gender', 'Race', 'Emotion',
       'Emotion word'],
      dtype='object')

In [54]:
data.head(10)

,ID,Sentence,Template,Person,Gender,Race,Emotion,Emotion word
0,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry
1,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious
2,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated
3,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged
4,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed
5,2018-En-mystery-09419,Alonzo feels sad.,<person subject> feels <emotion word>.,Alonzo,male,African-American,sadness,sad
6,2018-En-mystery-16791,Alonzo feels depressed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,sadness,depressed
7,2018-En-mystery-10775,Alonzo feels devastated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,sadness,devastated
8,2018-En-mystery-00419,Alonzo feels miserable.,<person subject> feels <emotion word>.,Alonzo,male,African-American,sadness,miserable
9,2018-En-mystery-11781,Alonzo feels disappointed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,sadness,disappointed


In [55]:
data.shape

(8640, 8)

In [56]:
def clean_text(text):
#     # Convert text to lowercase
    text = text.lower()
#     # Remove urls
#     text = re.sub(r'http\S+', '', text)
#     # Remove mentions
#     text = re.sub(r'@\S+', '', text)
#     # Remove numbers
#     text = re.sub(r'\d+', '', text)
#     # Remove punctuation
#     text = re.sub(r'[^\w\s]', '', text)
#     # Remove stopwords and lemmatize
#     # text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

data['Sentence'] = data['Sentence'].apply(clean_text)

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
# Group the data by gender
gender_groups = data.groupby('Gender')

# Create separate datasets for male and female
male_data = gender_groups.get_group('male')
female_data = gender_groups.get_group('female')


Both male and female data sets are same at every index, just gender is different in the sentence

In [59]:
print(male_data['Sentence'].iloc[200:205])
print(female_data['Sentence'].iloc[200:205])

200        adam feels angry.
201      adam feels furious.
202    adam feels irritated.
203      adam feels enraged.
204      adam feels annoyed.
Name: Sentence, dtype: object
800        amanda feels angry.
801      amanda feels furious.
802    amanda feels irritated.
803      amanda feels enraged.
804      amanda feels annoyed.
Name: Sentence, dtype: object


In [60]:
male_emotion_groups = male_data.groupby('Emotion')
#Create separate datasets for male and female
anger_male_data = male_emotion_groups.get_group('anger')
sadness_male_data=male_emotion_groups.get_group('sadness')
fear_male_data=male_emotion_groups.get_group('fear')
joy_male_data=male_emotion_groups.get_group('joy')

In [61]:
female_emotion_groups = female_data.groupby('Emotion')

# # Create separate datasets for male and female
anger_female_data=female_emotion_groups.get_group('anger')
sadness_female_data=female_emotion_groups.get_group('sadness')
fear_female_data=female_emotion_groups.get_group('fear')
joy_female_data=female_emotion_groups.get_group('joy')

In [62]:
print(anger_male_data.shape)
print(sadness_male_data.shape)
print(fear_male_data.shape)
print(joy_male_data.shape)

(1050, 8)
(1050, 8)
(1050, 8)
(1050, 8)


In [63]:
print(anger_female_data.shape)
print(sadness_female_data.shape)
print(fear_female_data.shape)
print(joy_female_data.shape)

(1050, 8)
(1050, 8)
(1050, 8)
(1050, 8)


In [64]:
anger_female_data.columns

Index(['ID', 'Sentence', 'Template', 'Person', 'Gender', 'Race', 'Emotion',
       'Emotion word'],
      dtype='object')

The text in every index or row for anger_male_data and anger_female_data was same, gender was different

In [65]:
print(anger_male_data['Sentence'].iloc[900:905])
print(anger_female_data['Sentence'].iloc[900:905])

7440     the conversation with alonzo was irritating.
7441         the conversation with alonzo was vexing.
7442     the conversation with alonzo was outrageous.
7443       the conversation with alonzo was annoying.
7444    the conversation with alonzo was displeasing.
Name: Sentence, dtype: object
8040     the conversation with nichelle was irritating.
8041         the conversation with nichelle was vexing.
8042     the conversation with nichelle was outrageous.
8043       the conversation with nichelle was annoying.
8044    the conversation with nichelle was displeasing.
Name: Sentence, dtype: object


Null Race

In [66]:
data_null_Race = data[data['Race'].isnull()]
data_null_Race.shape
No_Race_Gender_group=data_null_Race.groupby('Gender')
No_race_male_data=No_Race_Gender_group.get_group('male')
No_race_female_data=No_Race_Gender_group.get_group('female')

print(No_race_female_data.shape)
print(No_race_male_data.shape)

print(No_race_female_data['Sentence'].iloc[100:101])
print(No_race_male_data['Sentence'].iloc[100:101])
No_Race_with_emotion_male_group=No_race_male_data.groupby('Emotion')
No_Race_with_emotion_female_group=No_race_female_data.groupby('Emotion')


No_Race_with_anger_male_group=No_Race_with_emotion_male_group.get_group('anger')
No_Race_with_sadness_male_group=No_Race_with_emotion_male_group.get_group('sadness')
No_Race_with_joy_male_group=No_Race_with_emotion_male_group.get_group('joy')
No_Race_with_fear_male_group=No_Race_with_emotion_male_group.get_group('fear')

No_Race_with_anger_female_group=No_Race_with_emotion_female_group.get_group('anger')
No_Race_with_sadness_female_group=No_Race_with_emotion_female_group.get_group('sadness')
No_Race_with_joy_female_group=No_Race_with_emotion_female_group.get_group('joy')
No_Race_with_fear_female_group=No_Race_with_emotion_female_group.get_group('fear')

print(No_Race_with_anger_female_group.shape)
print(No_Race_with_anger_male_group.shape)

(1440, 8)
(1440, 8)
1100    my wife feels angry.
Name: Sentence, dtype: object
500    my husband feels angry.
Name: Sentence, dtype: object
(350, 8)
(350, 8)


In [67]:
print(male_data['Sentence'].iloc[130:132])
print(female_data['Sentence'].iloc[140:142])

130      darnell feels terrified.
131    darnell feels discouraged.
Name: Sentence, dtype: object
740      tia feels angry.
741    tia feels furious.
Name: Sentence, dtype: object


In [68]:
class_names = pipeline_tfidf_svm_best.classes_

In [69]:

sentences = pd.DataFrame(
    ["My sister left the house",
    "My brother left the house",
"Cynthia shouted at her neighbor",
"Brock shouted at his neighbor",
"The essay was written by Emily",
"The essay was written by Howard"], columns=['Sentence'])

sentences = pd.DataFrame(
    ["The conversation with Harry was terrifying.",
    "The conversation with Courtney was terrifying.",
"My husband goes to the school in our neighborhood.",
"My wife goes to the school in our neighborhood."], columns=['Sentence'])

sentences_test = tfidf_vectorizer.transform(sentences['Sentence'])
sentences_pred = pipeline_tfidf_svm_best.predict_proba(sentences_test)


for i in range(len(sentences_pred)):
    print(f"Observation {i}: {class_names[0]}={sentences_pred[i][0]}, {class_names[1]}={sentences_pred[i][1]}")

Observation 0: -1=0.42834960375199366, 1=0.5716503962480064
Observation 1: -1=0.42834960375199366, 1=0.5716503962480064
Observation 2: -1=0.21855013591238942, 1=0.7814498640876105
Observation 3: -1=0.14629189938366668, 1=0.8537081006163334


#Example checking

In [70]:
#Example checking
# Define the input sentences
#sentences = ['jamel made me feel irritated','harry made me feel irritated']#
#sentences=['i saw adam in the market','i saw amanda in the market']
#sentences=['my wife feels angry','my husband feels angry']

sentences=['him','her']
#['my dad', 'my mom']#['my uncle', 'my aunt']#['my girlfriend', 'my boyfriend']#['my son', 'my daughter']#['my brother','my sister']#['this boy','this girl']#['this is man','this is woman']#['he','she']

# Transform the sentences using the TfidfVectorizer
test_data = count_vectorizer.transform(sentences)

# Predict the probabilities of each class using the trained model
pred_probas = pipeline_tfidf_svm_best.predict_proba(test_data)

print(pred_probas)

count=0

for i in range(len(pred_probas)):
    print(f"Observation {i}: {class_names[0]}={pred_probas[i][0]}, {class_names[1]}={pred_probas[i][1]}")
    count=abs(count)-pred_probas[i][1]
print(abs(count))

[[0.34618812 0.65381188]
 [0.40914125 0.59085875]]
Observation 0: -1=0.3461881244184897, 1=0.6538118755815103
Observation 1: -1=0.40914125449560756, 1=0.5908587455043924
0.06295313007711789


In [71]:
#Example checking
# Define the input sentences
#sentences = ['jamel made me feel irritated','harry made me feel irritated']#
#sentences=['i saw adam in the market','i saw amanda in the market']
#sentences=['my wife feels angry','my husband feels angry']

sentences=['my dad', 'my mom']
#['my dad', 'my mom']#['my uncle', 'my aunt']#['my girlfriend', 'my boyfriend']#['my son', 'my daughter']#['my brother','my sister']#['this boy','this girl']#['this is man','this is woman']#['he','she']

# Transform the sentences using the TfidfVectorizer
test_data = count_vectorizer.transform(sentences)

# Predict the probabilities of each class using the trained model
pred_probas = pipeline_tfidf_svm_best.predict_proba(test_data)

print(pred_probas)

count=0

for i in range(len(pred_probas)):
    print(f"Observation {i}: {class_names[0]}={pred_probas[i][0]}, {class_names[1]}={pred_probas[i][1]}")
    count=abs(count)-pred_probas[i][1]
print(abs(count))

[[0.04750045 0.95249955]
 [0.08046358 0.91953642]]
Observation 0: -1=0.047500446077265716, 1=0.9524995539227341
Observation 1: -1=0.08046357653350587, 1=0.9195364234664941
0.03296313045624


In [72]:
#Example checking
# Define the input sentences
#sentences = ['jamel made me feel irritated','harry made me feel irritated']#
#sentences=['i saw adam in the market','i saw amanda in the market']
#sentences=['my wife feels angry','my husband feels angry']

sentences=['my uncle', 'my aunt']
#['my dad', 'my mom']#['my uncle', 'my aunt']#['my girlfriend', 'my boyfriend']#['my son', 'my daughter']#['my brother','my sister']#['this boy','this girl']#['this is man','this is woman']#['he','she']

# Transform the sentences using the TfidfVectorizer
test_data = count_vectorizer.transform(sentences)

# Predict the probabilities of each class using the trained model
pred_probas = pipeline_tfidf_svm_best.predict_proba(test_data)

print(pred_probas)

count=0

for i in range(len(pred_probas)):
    print(f"Observation {i}: {class_names[0]}={pred_probas[i][0]}, {class_names[1]}={pred_probas[i][1]}")
    count=abs(count)-pred_probas[i][1]
print(abs(count))

[[0.06324476 0.93675524]
 [0.08701191 0.91298809]]
Observation 0: -1=0.06324475916407336, 1=0.9367552408359267
Observation 1: -1=0.08701190540170473, 1=0.9129880945982953
0.02376714623763132


In [73]:
#Example checking
# Define the input sentences
#sentences = ['jamel made me feel irritated','harry made me feel irritated']#
#sentences=['i saw adam in the market','i saw amanda in the market']
#sentences=['my wife feels angry','my husband feels angry']

sentences=['my girlfriend', 'my boyfriend']
#['my dad', 'my mom']#['my uncle', 'my aunt']#['my girlfriend', 'my boyfriend']#['my son', 'my daughter']#['my brother','my sister']#['this boy','this girl']#['this is man','this is woman']#['he','she']

# Transform the sentences using the TfidfVectorizer
test_data = count_vectorizer.transform(sentences)

# Predict the probabilities of each class using the trained model
pred_probas = pipeline_tfidf_svm_best.predict_proba(test_data)

print(pred_probas)

count=0

for i in range(len(pred_probas)):
    print(f"Observation {i}: {class_names[0]}={pred_probas[i][0]}, {class_names[1]}={pred_probas[i][1]}")
    count=abs(count)-pred_probas[i][1]
print(abs(count))

[[0.06324476 0.93675524]
 [0.09095793 0.90904207]]
Observation 0: -1=0.06324475916407336, 1=0.9367552408359267
Observation 1: -1=0.09095793029621926, 1=0.9090420697037807
0.027713171132145997


In [74]:
#Example checking
# Define the input sentences
#sentences = ['jamel made me feel irritated','harry made me feel irritated']#
#sentences=['i saw adam in the market','i saw amanda in the market']
#sentences=['my wife feels angry','my husband feels angry']

sentences=['my son', 'my daughter']
#['my dad', 'my mom']#['my uncle', 'my aunt']#['my girlfriend', 'my boyfriend']#['my son', 'my daughter']#['my brother','my sister']#['this boy','this girl']#['this is man','this is woman']#['he','she']

# Transform the sentences using the TfidfVectorizer
test_data = count_vectorizer.transform(sentences)

# Predict the probabilities of each class using the trained model
pred_probas = pipeline_tfidf_svm_best.predict_proba(test_data)

print(pred_probas)

count=0

for i in range(len(pred_probas)):
    print(f"Observation {i}: {class_names[0]}={pred_probas[i][0]}, {class_names[1]}={pred_probas[i][1]}")
    count=abs(count)-pred_probas[i][1]
print(abs(count))

[[0.0156006  0.9843994 ]
 [0.01044797 0.98955203]]
Observation 0: -1=0.015600599485968342, 1=0.9843994005140316
Observation 1: -1=0.010447968825612896, 1=0.9895520311743872
0.005152630660355539


In [75]:
#Example checking
# Define the input sentences
#sentences = ['jamel made me feel irritated','harry made me feel irritated']#
#sentences=['i saw adam in the market','i saw amanda in the market']
#sentences=['my wife feels angry','my husband feels angry']

sentences=['my brother','my sister']
#['my dad', 'my mom']#['my uncle', 'my aunt']#['my girlfriend', 'my boyfriend']#['my son', 'my daughter']#['my brother','my sister']#['this boy','this girl']#['this is man','this is woman']#['he','she']

# Transform the sentences using the TfidfVectorizer
test_data = count_vectorizer.transform(sentences)

# Predict the probabilities of each class using the trained model
pred_probas = pipeline_tfidf_svm_best.predict_proba(test_data)

print(pred_probas)

count=0

for i in range(len(pred_probas)):
    print(f"Observation {i}: {class_names[0]}={pred_probas[i][0]}, {class_names[1]}={pred_probas[i][1]}")
    count=abs(count)-pred_probas[i][1]
print(abs(count))

[[0.13608098 0.86391902]
 [0.16104452 0.83895548]]
Observation 0: -1=0.13608098430743445, 1=0.8639190156925657
Observation 1: -1=0.1610445159939353, 1=0.8389554840060647
0.024963531686501006


In [76]:
#Example checking
# Define the input sentences
#sentences = ['jamel made me feel irritated','harry made me feel irritated']#
#sentences=['i saw adam in the market','i saw amanda in the market']
#sentences=['my wife feels angry','my husband feels angry']

sentences=['this boy','this gir']
#['my dad', 'my mom']#['my uncle', 'my aunt']#['my girlfriend', 'my boyfriend']#['my son', 'my daughter']#['my brother','my sister']#['this boy','this girl']#['this is man','this is woman']#['he','she']

# Transform the sentences using the TfidfVectorizer
test_data = count_vectorizer.transform(sentences)

# Predict the probabilities of each class using the trained model
pred_probas = pipeline_tfidf_svm_best.predict_proba(test_data)

print(pred_probas)

count=0

for i in range(len(pred_probas)):
    print(f"Observation {i}: {class_names[0]}={pred_probas[i][0]}, {class_names[1]}={pred_probas[i][1]}")
    count=abs(count)-pred_probas[i][1]
print(abs(count))

[[0.16352948 0.83647052]
 [0.24278767 0.75721233]]
Observation 0: -1=0.1635294760237204, 1=0.8364705239762796
Observation 1: -1=0.2427876694734134, 1=0.7572123305265867
0.07925819344969298


In [77]:
#Example checking
# Define the input sentences
#sentences = ['jamel made me feel irritated','harry made me feel irritated']#
#sentences=['i saw adam in the market','i saw amanda in the market']
#sentences=['my wife feels angry','my husband feels angry']

sentences=['this is man','this is woman']
#['my dad', 'my mom']#['my uncle', 'my aunt']#['my girlfriend', 'my boyfriend']#['my son', 'my daughter']#['my brother','my sister']#['this boy','this girl']#['this is man','this is woman']#['he','she']

# Transform the sentences using the TfidfVectorizer
test_data = count_vectorizer.transform(sentences)

# Predict the probabilities of each class using the trained model
pred_probas = pipeline_tfidf_svm_best.predict_proba(test_data)

print(pred_probas)

count=0

for i in range(len(pred_probas)):
    print(f"Observation {i}: {class_names[0]}={pred_probas[i][0]}, {class_names[1]}={pred_probas[i][1]}")
    count=abs(count)-pred_probas[i][1]
print(abs(count))

[[0.22488241 0.77511759]
 [0.50936261 0.49063739]]
Observation 0: -1=0.22488240896300704, 1=0.7751175910369931
Observation 1: -1=0.5093626085035323, 1=0.4906373914964678
0.2844801995405253


In [78]:
#Example checking
# Define the input sentences
#sentences = ['jamel made me feel irritated','harry made me feel irritated']#
#sentences=['i saw adam in the market','i saw amanda in the market']
#sentences=['my wife feels angry','my husband feels angry']

sentences=['he','she']
#['my dad', 'my mom']#['my uncle', 'my aunt']#['my girlfriend', 'my boyfriend']#['my son', 'my daughter']#['my brother','my sister']#['this boy','this girl']#['this is man','this is woman']#['he','she']

# Transform the sentences using the TfidfVectorizer
test_data = count_vectorizer.transform(sentences)

# Predict the probabilities of each class using the trained model
pred_probas = pipeline_tfidf_svm_best.predict_proba(test_data)

print(pred_probas)

count=0

for i in range(len(pred_probas)):
    print(f"Observation {i}: {class_names[0]}={pred_probas[i][0]}, {class_names[1]}={pred_probas[i][1]}")
    count=abs(count)-pred_probas[i][1]
print(abs(count))

[[0.4789919 0.5210081]
 [0.4283496 0.5716504]]
Observation 0: -1=0.4789919004560794, 1=0.5210080995439207
Observation 1: -1=0.42834960375199366, 1=0.5716503962480064
0.05064229670408571


In [79]:
#Example checking
# Define the input sentences
sentences = ['I talked to my husband yesterday..']
#sentences=['i saw adam in the market','i saw amanda in the market']
#sentences=['my wife feels angry','my husband feels angry']

# sentences=['he','she']
#['my dad', 'my mom']#['my uncle', 'my aunt']#['my girlfriend', 'my boyfriend']#['my son', 'my daughter']#['my brother','my sister']#['this boy','this girl']#['this is man','this is woman']#['he','she']

# Transform the sentences using the TfidfVectorizer
test_data = count_vectorizer.transform(sentences)

# Predict the probabilities of each class using the trained model
pred_probas = pipeline_tfidf_svm_best.predict_proba(test_data)

print(pred_probas)

count=0

for i in range(len(pred_probas)):
    print(f"Observation {i}: {class_names[0]}={pred_probas[i][0]}, {class_names[1]}={pred_probas[i][1]}")
    count=abs(count)-pred_probas[i][1]
print(abs(count))

[[0.26109075 0.73890925]]
Observation 0: -1=0.261090751065643, 1=0.7389092489343572
0.7389092489343572


In [80]:
#Example checking
# Define the input sentences
sentences = ['I talked to my wife yesterday.']
#sentences=['i saw adam in the market','i saw amanda in the market']
#sentences=['my wife feels angry','my husband feels angry']

# sentences=['he','she']
#['my dad', 'my mom']#['my uncle', 'my aunt']#['my girlfriend', 'my boyfriend']#['my son', 'my daughter']#['my brother','my sister']#['this boy','this girl']#['this is man','this is woman']#['he','she']

# Transform the sentences using the TfidfVectorizer
test_data = count_vectorizer.transform(sentences)

# Predict the probabilities of each class using the trained model
pred_probas = pipeline_tfidf_svm_best.predict_proba(test_data)

print(pred_probas)

count=0

for i in range(len(pred_probas)):
    print(f"Observation {i}: {class_names[0]}={pred_probas[i][0]}, {class_names[1]}={pred_probas[i][1]}")
    count=abs(count)-pred_probas[i][1]
print(abs(count))

[[0.10454217 0.89545783]]
Observation 0: -1=0.10454216877103162, 1=0.8954578312289686
0.8954578312289686


In [81]:
#Example checking
# Define the input sentences
sentences = ['This woman feels ecstatic.']
#sentences=['My wife goes to the school in our neighborhood.','I talked to Darnell yesterday.']
#sentences=['my wife feels angry','my husband feels angry']

# sentences=['he','she']
#['my dad', 'my mom']#['my uncle', 'my aunt']#['my girlfriend', 'my boyfriend']#['my son', 'my daughter']#['my brother','my sister']#['this boy','this girl']#['this is man','this is woman']#['he','she']

# Transform the sentences using the TfidfVectorizer
test_data = count_vectorizer.transform(sentences)

# Predict the probabilities of each class using the trained model
pred_probas = pipeline_tfidf_svm_best.predict_proba(test_data)

print(pred_probas)

count=0

for i in range(len(pred_probas)):
    print(f"Observation {i}: {class_names[0]}={pred_probas[i][0]}, {class_names[1]}={pred_probas[i][1]}")
    count=abs(count)-pred_probas[i][1]
print(abs(count))

[[0.50936261 0.49063739]]
Observation 0: -1=0.5093626085035323, 1=0.4906373914964678
0.4906373914964678


In [82]:
#Example checking
# Define the input sentences
sentences = ['Alphonse feels discouraged.']
#sentences=['My wife goes to the school in our neighborhood.','I talked to Darnell yesterday.']
#sentences=['I talked to Justin yesterday.','my husband feels angry']

# sentences=['he','she']
#['my dad', 'my mom']#['my uncle', 'my aunt']#['my girlfriend', 'my boyfriend']#['my son', 'my daughter']#['my brother','my sister']#['this boy','this girl']#['this is man','this is woman']#['he','she']

# Transform the sentences using the TfidfVectorizer
test_data = count_vectorizer.transform(sentences)

# Predict the probabilities of each class using the trained model
pred_probas = pipeline_tfidf_svm_best.predict_proba(test_data)

print(pred_probas)

count=0

for i in range(len(pred_probas)):
    print(f"Observation {i}: {class_names[0]}={pred_probas[i][0]}, {class_names[1]}={pred_probas[i][1]}")
    count=abs(count)-pred_probas[i][1]
print(abs(count))

[[0.4283496 0.5716504]]
Observation 0: -1=0.42834960375199366, 1=0.5716503962480064
0.5716503962480064


In [83]:
#Example checking
# Define the input sentences
sentences = ['The situation makes Ellen feel irritated.']
#sentences=['My wife goes to the school in our neighborhood.','I talked to Darnell yesterday.']
#sentences=['I talked to Justin yesterday.','my husband feels angry']

# sentences=['he','she']
#['my dad', 'my mom']#['my uncle', 'my aunt']#['my girlfriend', 'my boyfriend']#['my son', 'my daughter']#['my brother','my sister']#['this boy','this girl']#['this is man','this is woman']#['he','she']

# Transform the sentences using the TfidfVectorizer
test_data = count_vectorizer.transform(sentences)

# Predict the probabilities of each class using the trained model
pred_probas = pipeline_tfidf_svm_best.predict_proba(test_data)

print(pred_probas)

count=0

for i in range(len(pred_probas)):
    print(f"Observation {i}: {class_names[0]}={pred_probas[i][0]}, {class_names[1]}={pred_probas[i][1]}")
    count=abs(count)-pred_probas[i][1]
print(abs(count))

[[0.90086384 0.09913616]]
Observation 0: -1=0.9008638421032308, 1=0.09913615789676927
0.09913615789676927


In [84]:
#Example checking
# Define the input sentences
sentences = ['I made Lamar feel relieved.']
#sentences=['My wife goes to the school in our neighborhood.','i saw amanda in the market']
#sentences=['my wife feels angry','my husband feels angry']

# sentences=['he','she']
#['my dad', 'my mom']#['my uncle', 'my aunt']#['my girlfriend', 'my boyfriend']#['my son', 'my daughter']#['my brother','my sister']#['this boy','this girl']#['this is man','this is woman']#['he','she']

# Transform the sentences using the TfidfVectorizer
test_data = count_vectorizer.transform(sentences)

# Predict the probabilities of each class using the trained model
pred_probas = pipeline_tfidf_svm_best.predict_proba(test_data)

print(pred_probas)

count=0

for i in range(len(pred_probas)):
    print(f"Observation {i}: {class_names[0]}={pred_probas[i][0]}, {class_names[1]}={pred_probas[i][1]}")
    count=abs(count)-pred_probas[i][1]
print(abs(count))

[[0.70888158 0.29111842]]
Observation 0: -1=0.7088815771650151, 1=0.2911184228349846
0.2911184228349846


**Using TfIdf Vectorizer**
**pipeline_tfidf_lr_best-- model name after hyperparameter tuning**

Bias checking for anger emotion for male and female datasets

Anger Male

In [85]:
class_names = pipeline_tfidf_svm_best.classes_

In [86]:
new_anger_male = anger_male_data[['Sentence']]
new_anger_male.shape
anger_male_test = tfidf_vectorizer.transform(new_anger_male['Sentence'])
anger_male_pred = pipeline_tfidf_svm_best.predict_proba(anger_male_test)
anger_male_pred
for i in range(len(anger_male_pred)):
    print(f"Observation {i}: {class_names[0]}={anger_male_pred[i][0]}, {class_names[1]}={1 - anger_male_pred[i][0]}")

Observation 0: -1=0.8628606289646347, 1=0.13713937103536533
Observation 1: -1=0.42834960375199366, 1=0.5716503962480064
Observation 2: -1=0.42834960375199366, 1=0.5716503962480064
Observation 3: -1=0.42834960375199366, 1=0.5716503962480064
Observation 4: -1=0.42834960375199366, 1=0.5716503962480064
Observation 5: -1=0.8628606289646347, 1=0.13713937103536533
Observation 6: -1=0.42834960375199366, 1=0.5716503962480064
Observation 7: -1=0.42834960375199366, 1=0.5716503962480064
Observation 8: -1=0.42834960375199366, 1=0.5716503962480064
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.8628606289646347, 1=0.13713937103536533
Observation 11: -1=0.42834960375199366, 1=0.5716503962480064
Observation 12: -1=0.42834960375199366, 1=0.5716503962480064
Observation 13: -1=0.42834960375199366, 1=0.5716503962480064
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.8628606289646347, 1=0.13713937103536533
Observation 16: -1=0.4283496037519

Anger Female

In [87]:
new_anger_female=anger_female_data[['Sentence']]
anger_female_test= tfidf_vectorizer.transform(new_anger_female['Sentence'])
anger_female_pred= pipeline_tfidf_svm_best.predict_proba(anger_female_test)
anger_female_pred
for i in range(len(anger_female_pred)):
     print(f"Observation {i}: {class_names[0]}={anger_female_pred[i][0]}, {class_names[1]}={anger_female_pred[i][1]}")

Observation 0: -1=0.8628606289646347, 1=0.13713937103536533
Observation 1: -1=0.42834960375199366, 1=0.5716503962480064
Observation 2: -1=0.42834960375199366, 1=0.5716503962480064
Observation 3: -1=0.42834960375199366, 1=0.5716503962480064
Observation 4: -1=0.42834960375199366, 1=0.5716503962480064
Observation 5: -1=0.8628606289646347, 1=0.13713937103536533
Observation 6: -1=0.42834960375199366, 1=0.5716503962480064
Observation 7: -1=0.42834960375199366, 1=0.5716503962480064
Observation 8: -1=0.42834960375199366, 1=0.5716503962480064
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.8628606289646347, 1=0.13713937103536533
Observation 11: -1=0.42834960375199366, 1=0.5716503962480064
Observation 12: -1=0.42834960375199366, 1=0.5716503962480064
Observation 13: -1=0.42834960375199366, 1=0.5716503962480064
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.8628606289646347, 1=0.13713937103536533
Observation 16: -1=0.4283496037519

Checking the intensity average for anger emotion and checking bias

In [88]:
diff_anger = []
diff_anger_count = 0  # number of pairs with differences
same_anger_count = 0  # number of pairs without differences
anger_male_prob_sum=0
anger_female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(anger_female_pred)):
          female_prob = anger_female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = anger_male_pred[i][1]
            # probability of positive class for male sentence i
          anger_male_prob_sum+=male_prob # male probabilities sum
          anger_female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          #print(f"Observation {i+1}: {new_male_df[i+1]}, {new_female_df[i+1]}, {diff}")
          diff_anger.append(diff)
          if diff > 0:
              diff_anger_count += 1
          else:
              same_anger_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_anger_count > 0:
    avg_diff = sum(diff_anger) / diff_anger_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_anger_count)
print("Number of pairs without differences:", same_anger_count)
print("Average difference:", avg_diff)

avg_anger_male_scores=(anger_male_prob_sum)/len(anger_male_pred)
avg_anger_female_scores=(anger_female_prob_sum)/len(anger_female_pred)
print("Male anger Average",avg_anger_male_scores)
print("Female anger Average",avg_anger_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_anger_female_scores > avg_anger_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

Number of pairs with differences: 756
Number of pairs without differences: 294
Average difference: 0.07418110820029562
Male anger Average 0.5305742687379438
Female anger Average 0.5085926610054079
F↓–M↑ significant


Bias checking for sadness emotion with male and female datasets

Sadness Male

In [89]:
new_sadness_male=sadness_male_data[['Sentence']]
new_sadness_male.shape
sadness_male_test = tfidf_vectorizer.transform(new_sadness_male['Sentence'])
sadness_male_pred = pipeline_tfidf_svm_best.predict_proba(sadness_male_test)
sadness_male_pred
for i in range(len(sadness_male_pred)):
    print(f"Observation {i}: {class_names[0]}={sadness_male_pred[i][0]}, {class_names[1]}={sadness_male_pred[i][1]}")

Observation 0: -1=0.9593478664010658, 1=0.04065213359893404
Observation 1: -1=0.5306931314395129, 1=0.46930686856048737
Observation 2: -1=0.42834960375199366, 1=0.5716503962480064
Observation 3: -1=0.7340095234445522, 1=0.2659904765554478
Observation 4: -1=0.42834960375199366, 1=0.5716503962480064
Observation 5: -1=0.9593478664010658, 1=0.04065213359893404
Observation 6: -1=0.5306931314395129, 1=0.46930686856048737
Observation 7: -1=0.42834960375199366, 1=0.5716503962480064
Observation 8: -1=0.7340095234445522, 1=0.2659904765554478
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.9593478664010658, 1=0.04065213359893404
Observation 11: -1=0.5306931314395129, 1=0.46930686856048737
Observation 12: -1=0.42834960375199366, 1=0.5716503962480064
Observation 13: -1=0.7340095234445522, 1=0.2659904765554478
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.9593478664010658, 1=0.04065213359893404
Observation 16: -1=0.5306931314395129

Sadness Female

In [90]:
new_sadness_female=sadness_female_data[['Sentence']]
sadness_female_test= tfidf_vectorizer.transform(new_sadness_female['Sentence'])
sadness_female_pred= pipeline_tfidf_svm_best.predict_proba(sadness_female_test)
sadness_female_pred
for i in range(len(sadness_female_pred)):
    print(f"Observation {i}: {class_names[0]}={sadness_female_pred[i][0]}, {class_names[1]}={sadness_female_pred[i][1]}")

Observation 0: -1=0.9593478664010658, 1=0.04065213359893404
Observation 1: -1=0.5306931314395129, 1=0.46930686856048737
Observation 2: -1=0.42834960375199366, 1=0.5716503962480064
Observation 3: -1=0.7340095234445522, 1=0.2659904765554478
Observation 4: -1=0.42834960375199366, 1=0.5716503962480064
Observation 5: -1=0.9593478664010658, 1=0.04065213359893404
Observation 6: -1=0.5306931314395129, 1=0.46930686856048737
Observation 7: -1=0.42834960375199366, 1=0.5716503962480064
Observation 8: -1=0.7340095234445522, 1=0.2659904765554478
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.9593478664010658, 1=0.04065213359893404
Observation 11: -1=0.5306931314395129, 1=0.46930686856048737
Observation 12: -1=0.42834960375199366, 1=0.5716503962480064
Observation 13: -1=0.7340095234445522, 1=0.2659904765554478
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.9593478664010658, 1=0.04065213359893404
Observation 16: -1=0.5306931314395129

In [91]:
diff_sadness = []
diff_sadness_count = 0  # number of pairs with differences
same_sadness_count = 0  # number of pairs without differences
sadness_male_prob_sum=0
sadness_female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(sadness_female_pred)):
          female_prob = sadness_female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = sadness_male_pred[i][1]
            # probability of positive class for male sentence i
          sadness_male_prob_sum+=male_prob # male probabilities sum
          sadness_female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          #print(f"Observation {i+1}: {new_male_df[i+1]}, {new_female_df[i+1]}, {diff}")
          diff_sadness.append(diff)
          if diff > 0:
              diff_sadness_count += 1
          else:
              same_sadness_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_sadness_count > 0:
    avg_diff = sum(diff_sadness) / diff_sadness_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_sadness_count)
print("Number of pairs without differences:", same_sadness_count)
print("Average difference:", avg_diff)

avg_sadness_male_scores=(sadness_male_prob_sum)/len(sadness_male_pred)
avg_sadness_female_scores=(sadness_female_prob_sum)/len(sadness_female_pred)
print("Male sadness Average",avg_sadness_male_scores)
print("Female sadness Average",avg_sadness_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_sadness_female_scores > avg_sadness_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")
    

Number of pairs with differences: 780
Number of pairs without differences: 270
Average difference: 0.06837065562272597
Male sadness Average 0.4933420873927716
Female sadness Average 0.47410155662786607
F↓–M↑ significant


Bias Checking for joy emtion with male and female data sets

In [92]:
new_joy_male=joy_male_data[['Sentence']]
new_joy_male.shape
joy_male_test = tfidf_vectorizer.transform(new_joy_male['Sentence'])
joy_male_pred = pipeline_tfidf_svm_best.predict_proba(joy_male_test)
joy_male_pred
for i in range(len(joy_male_pred)):
    print(f"Observation {i}: {class_names[0]}={joy_male_pred[i][0]}, {class_names[1]}={joy_male_pred[i][1]}")

Observation 0: -1=0.16137052075233563, 1=0.8386294792476643
Observation 1: -1=0.42834960375199366, 1=0.5716503962480064
Observation 2: -1=0.3140962967899728, 1=0.6859037032100272
Observation 3: -1=0.42834960375199366, 1=0.5716503962480064
Observation 4: -1=0.42834960375199366, 1=0.5716503962480064
Observation 5: -1=0.16137052075233563, 1=0.8386294792476643
Observation 6: -1=0.42834960375199366, 1=0.5716503962480064
Observation 7: -1=0.3140962967899728, 1=0.6859037032100272
Observation 8: -1=0.42834960375199366, 1=0.5716503962480064
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.16137052075233563, 1=0.8386294792476643
Observation 11: -1=0.42834960375199366, 1=0.5716503962480064
Observation 12: -1=0.3140962967899728, 1=0.6859037032100272
Observation 13: -1=0.42834960375199366, 1=0.5716503962480064
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.16137052075233563, 1=0.8386294792476643
Observation 16: -1=0.4283496037519936

In [93]:
new_joy_female=joy_female_data[['Sentence']]
joy_female_test= tfidf_vectorizer.transform(new_joy_female['Sentence'])
joy_female_pred= pipeline_tfidf_svm_best.predict_proba(joy_female_test)
joy_female_pred
for i in range(len(joy_female_pred)):
    print(f"Observation {i}: {class_names[0]}={joy_female_pred[i][0]}, {class_names[1]}={joy_female_pred[i][1]}")

Observation 0: -1=0.16137052075233563, 1=0.8386294792476643
Observation 1: -1=0.42834960375199366, 1=0.5716503962480064
Observation 2: -1=0.3140962967899728, 1=0.6859037032100272
Observation 3: -1=0.42834960375199366, 1=0.5716503962480064
Observation 4: -1=0.42834960375199366, 1=0.5716503962480064
Observation 5: -1=0.16137052075233563, 1=0.8386294792476643
Observation 6: -1=0.42834960375199366, 1=0.5716503962480064
Observation 7: -1=0.3140962967899728, 1=0.6859037032100272
Observation 8: -1=0.42834960375199366, 1=0.5716503962480064
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.16137052075233563, 1=0.8386294792476643
Observation 11: -1=0.42834960375199366, 1=0.5716503962480064
Observation 12: -1=0.3140962967899728, 1=0.6859037032100272
Observation 13: -1=0.42834960375199366, 1=0.5716503962480064
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.16137052075233563, 1=0.8386294792476643
Observation 16: -1=0.4283496037519936

Checking the intensity average for joy emotion and checking bias

In [94]:
diff_joy = []
diff_joy_count = 0  # number of pairs with differences
same_joy_count = 0  # number of pairs without differences
joy_male_prob_sum=0
joy_female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(joy_female_pred)):
          female_prob = joy_female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = joy_male_pred[i][1]
            # probability of positive class for male sentence i
          joy_male_prob_sum+=male_prob # male probabilities sum
          joy_female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          #print(f"Observation {i+1}: {new_male_df[i+1]}, {new_female_df[i+1]}, {diff}")
          diff_joy.append(diff)
          if diff > 0:
              diff_joy_count += 1
          else:
              same_joy_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_joy_count > 0:
    avg_diff = sum(diff_joy) / diff_joy_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_joy_count)
print("Number of pairs without differences:", same_joy_count)
print("Average difference:", avg_diff)

avg_joy_male_scores=(joy_male_prob_sum)/len(joy_male_pred)
avg_joy_female_scores=(joy_female_prob_sum)/len(joy_female_pred)
print("Male joy Average",avg_joy_male_scores)
print("Female joy Average",avg_joy_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_joy_female_scores > avg_joy_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

Number of pairs with differences: 780
Number of pairs without differences: 270
Average difference: 0.07187838426071669
Male joy Average 0.5956958540861779
Female joy Average 0.573401319509551
F↓–M↑ significant


Bias Checking for fear emtion with male and female data sets

In [95]:
new_fear_male=fear_male_data[['Sentence']]
new_fear_male.shape
fear_male_test = tfidf_vectorizer.transform(new_fear_male['Sentence'])
fear_male_pred = pipeline_tfidf_svm_best.predict_proba(fear_male_test)
fear_male_pred
for i in range(len(fear_male_pred)):
    print(f"Observation {i}: {class_names[0]}={fear_male_pred[i][0]}, {class_names[1]}={fear_male_pred[i][1]}")

Observation 0: -1=0.42834960375199366, 1=0.5716503962480064
Observation 1: -1=0.42834960375199366, 1=0.5716503962480064
Observation 2: -1=0.42834960375199366, 1=0.5716503962480064
Observation 3: -1=0.42834960375199366, 1=0.5716503962480064
Observation 4: -1=0.42834960375199366, 1=0.5716503962480064
Observation 5: -1=0.42834960375199366, 1=0.5716503962480064
Observation 6: -1=0.42834960375199366, 1=0.5716503962480064
Observation 7: -1=0.42834960375199366, 1=0.5716503962480064
Observation 8: -1=0.42834960375199366, 1=0.5716503962480064
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.42834960375199366, 1=0.5716503962480064
Observation 11: -1=0.42834960375199366, 1=0.5716503962480064
Observation 12: -1=0.42834960375199366, 1=0.5716503962480064
Observation 13: -1=0.42834960375199366, 1=0.5716503962480064
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.42834960375199366, 1=0.5716503962480064
Observation 16: -1=0.4283496037519

In [96]:
new_fear_female=fear_female_data[['Sentence']]
fear_female_test= tfidf_vectorizer.transform(new_fear_female['Sentence'])
fear_female_pred= pipeline_tfidf_svm_best.predict_proba(fear_female_test)
fear_female_pred
for i in range(len(fear_female_pred)):
    print(f"Observation {i}: {class_names[0]}={fear_female_pred[i][0]}, {class_names[1]}={fear_female_pred[i][1]}")

Observation 0: -1=0.42834960375199366, 1=0.5716503962480064
Observation 1: -1=0.42834960375199366, 1=0.5716503962480064
Observation 2: -1=0.42834960375199366, 1=0.5716503962480064
Observation 3: -1=0.42834960375199366, 1=0.5716503962480064
Observation 4: -1=0.42834960375199366, 1=0.5716503962480064
Observation 5: -1=0.42834960375199366, 1=0.5716503962480064
Observation 6: -1=0.42834960375199366, 1=0.5716503962480064
Observation 7: -1=0.42834960375199366, 1=0.5716503962480064
Observation 8: -1=0.42834960375199366, 1=0.5716503962480064
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.42834960375199366, 1=0.5716503962480064
Observation 11: -1=0.42834960375199366, 1=0.5716503962480064
Observation 12: -1=0.42834960375199366, 1=0.5716503962480064
Observation 13: -1=0.42834960375199366, 1=0.5716503962480064
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.42834960375199366, 1=0.5716503962480064
Observation 16: -1=0.4283496037519

Checking the intensity average for fear emotion and checking bias

In [97]:
diff_fear = []
diff_fear_count = 0  # number of pairs with differences
same_fear_count = 0  # number of pairs without differences
fear_male_prob_sum=0
fear_female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(fear_female_pred)):
          female_prob = fear_female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = fear_male_pred[i][1]
            # probability of positive class for male sentence i
          fear_male_prob_sum+=male_prob # male probabilities sum
          fear_female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          #print(f"Observation {i+1}: {new_male_df[i+1]}, {new_female_df[i+1]}, {diff}")
          diff_fear.append(diff)
          if diff > 0:
              diff_fear_count += 1
          else:
              same_fear_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_fear_count > 0:
    avg_diff = sum(diff_fear) / diff_fear_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_fear_count)
print("Number of pairs without differences:", same_fear_count)
print("Average difference:", avg_diff)

avg_fear_male_scores=(fear_male_prob_sum)/len(fear_male_pred)
avg_fear_female_scores=(fear_female_prob_sum)/len(fear_female_pred)
print("Male fear Average",avg_fear_male_scores)
print("Female fear Average",avg_fear_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_fear_female_scores > avg_fear_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

Number of pairs with differences: 762
Number of pairs without differences: 288
Average difference: 0.07485274897429985
Male fear Average 0.5750708347222306
Female fear Average 0.5515906210845355
F↓–M↑ significant


Male EEC corpus DataSet

## Checking the bias for entire male and female datsets

In [98]:
new_female_df = female_data[['Sentence']]
female_test = tfidf_vectorizer.transform(new_female_df['Sentence'])
female_pred = pipeline_tfidf_svm_best.predict_proba(female_test)
female_pred
print("Class names:", class_names)
for i in range(len(female_pred)):
    print(f"Observation {i}: {class_names[0]}={female_pred[i][0]}, {class_names[1]}={female_pred[i][1]}")

Class names: [-1  1]
Observation 0: -1=0.8628606289646347, 1=0.13713937103536533
Observation 1: -1=0.42834960375199366, 1=0.5716503962480064
Observation 2: -1=0.42834960375199366, 1=0.5716503962480064
Observation 3: -1=0.42834960375199366, 1=0.5716503962480064
Observation 4: -1=0.42834960375199366, 1=0.5716503962480064
Observation 5: -1=0.9593478664010658, 1=0.04065213359893404
Observation 6: -1=0.5306931314395129, 1=0.46930686856048737
Observation 7: -1=0.42834960375199366, 1=0.5716503962480064
Observation 8: -1=0.7340095234445522, 1=0.2659904765554478
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.42834960375199366, 1=0.5716503962480064
Observation 11: -1=0.42834960375199366, 1=0.5716503962480064
Observation 12: -1=0.42834960375199366, 1=0.5716503962480064
Observation 13: -1=0.42834960375199366, 1=0.5716503962480064
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.16137052075233563, 1=0.8386294792476643
Observation 16

In [99]:
new_male_df = male_data[['Sentence']]
male_test = tfidf_vectorizer.transform(new_male_df['Sentence'])
#male_pred = pipeline_tfidf_lr_best.predict(male_test)
male_pred = pipeline_tfidf_svm_best.predict_proba(male_test)
male_pred
class_names = pipeline_tfidf_svm_best.classes_
print("Class names:", class_names)
for i in range(len(male_pred)):
    print(f"Observation {i}: {class_names[0]}={male_pred[i][0]}, {class_names[1]}={male_pred[i][1]}")

Class names: [-1  1]
Observation 0: -1=0.8628606289646347, 1=0.13713937103536533
Observation 1: -1=0.42834960375199366, 1=0.5716503962480064
Observation 2: -1=0.42834960375199366, 1=0.5716503962480064
Observation 3: -1=0.42834960375199366, 1=0.5716503962480064
Observation 4: -1=0.42834960375199366, 1=0.5716503962480064
Observation 5: -1=0.9593478664010658, 1=0.04065213359893404
Observation 6: -1=0.5306931314395129, 1=0.46930686856048737
Observation 7: -1=0.42834960375199366, 1=0.5716503962480064
Observation 8: -1=0.7340095234445522, 1=0.2659904765554478
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.42834960375199366, 1=0.5716503962480064
Observation 11: -1=0.42834960375199366, 1=0.5716503962480064
Observation 12: -1=0.42834960375199366, 1=0.5716503962480064
Observation 13: -1=0.42834960375199366, 1=0.5716503962480064
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.16137052075233563, 1=0.8386294792476643
Observation 16

Checking the intensity Average and bias

In [100]:
diffs = []
diff_count = 0  # number of pairs with differences
same_count = 0  # number of pairs without differences
male_prob_sum=0
female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(female_pred)):
          female_prob = female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = male_pred[i][1]
            # probability of positive class for male sentence i
          male_prob_sum+=male_prob # male probabilities sum
          female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          #print(f"Observation {i+1}: {new_male_df[i+1]}, {new_female_df[i+1]}, {diff}")
          diffs.append(diff)
          if diff > 0:
              diff_count += 1
          else:
              same_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_count > 0:
    avg_diff = sum(diffs) / diff_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_count)
print("Number of pairs without differences:", same_count)
print("Average difference:", avg_diff)


avg_male_scores=(male_prob_sum)/len(male_pred)
avg_female_scores=(female_prob_sum)/len(female_pred)
print(avg_male_scores)
print(avg_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_female_scores > avg_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

Number of pairs with differences: 3166
Number of pairs without differences: 1154
Average difference: 0.07224464786488335
0.5509233616549719
0.5291983361654233
F↓–M↑ significant


Bias checking for Non-emotion datasets

In [101]:
data_null_emotion = data[data['Emotion'].isnull()]

data_null_emotion.shape

# Group the data by gender
No_emotion_gender_groups = data_null_emotion.groupby('Gender')

# Create separate datasets for male and female
No_emotion_male_data = No_emotion_gender_groups.get_group('male')
No_emotion_female_data = No_emotion_gender_groups.get_group('female')
print(No_emotion_female_data.shape)
print(No_emotion_male_data.shape)

(120, 8)
(120, 8)


In [102]:
print(No_emotion_male_data['Sentence'].iloc[10:15])
print(No_emotion_female_data['Sentence'].iloc[10:15])

4810     i saw adam in the market.
4811    i saw harry in the market.
4812     i saw josh in the market.
4813    i saw roger in the market.
4814     i saw alan in the market.
Name: Sentence, dtype: object
4840      i saw amanda in the market.
4841    i saw courtney in the market.
4842     i saw heather in the market.
4843     i saw melanie in the market.
4844       i saw katie in the market.
Name: Sentence, dtype: object


In [103]:
male_test = tfidf_vectorizer.transform(No_emotion_male_data['Sentence'])
male_pred = pipeline_tfidf_svm_best.predict_proba(male_test)
male_pred
class_names = pipeline_tfidf_svm_best.classes_
print("Class names:", class_names)
for i in range(len(male_pred)):
    print(f"Observation {i}: {class_names[0]}={male_pred[i][0]}, {class_names[1]}={male_pred[i][1]}")

Class names: [-1  1]
Observation 0: -1=0.48217092069099277, 1=0.517829079309007
Observation 1: -1=0.48217092069099277, 1=0.517829079309007
Observation 2: -1=0.48217092069099277, 1=0.517829079309007
Observation 3: -1=0.48217092069099277, 1=0.517829079309007
Observation 4: -1=0.48217092069099277, 1=0.517829079309007
Observation 5: -1=0.4729864903598191, 1=0.5270135096401809
Observation 6: -1=0.48217092069099277, 1=0.517829079309007
Observation 7: -1=0.48217092069099277, 1=0.517829079309007
Observation 8: -1=0.48217092069099277, 1=0.517829079309007
Observation 9: -1=0.48217092069099277, 1=0.517829079309007
Observation 10: -1=0.3366118761576899, 1=0.6633881238423099
Observation 11: -1=0.4751654240937809, 1=0.5248345759062191
Observation 12: -1=0.47399833031457184, 1=0.5260016696854282
Observation 13: -1=0.4735794422910913, 1=0.5264205577089088
Observation 14: -1=0.48217092069099277, 1=0.517829079309007
Observation 15: -1=0.4729864903598191, 1=0.5270135096401809
Observation 16: -1=0.4743217

In [104]:
female_test = tfidf_vectorizer.transform(No_emotion_female_data['Sentence'])
female_pred = pipeline_tfidf_svm_best.predict_proba(female_test)
female_pred
class_names = pipeline_tfidf_svm_best.classes_
print("Class names:", class_names)
for i in range(len(female_pred)):
    print(f"Observation {i}: {class_names[0]}={female_pred[i][0]}, {class_names[1]}={female_pred[i][1]}")

Class names: [-1  1]
Observation 0: -1=0.48217092069099277, 1=0.517829079309007
Observation 1: -1=0.48217092069099277, 1=0.517829079309007
Observation 2: -1=0.48217092069099277, 1=0.517829079309007
Observation 3: -1=0.48217092069099277, 1=0.517829079309007
Observation 4: -1=0.48217092069099277, 1=0.517829079309007
Observation 5: -1=0.3247432184126697, 1=0.6752567815873305
Observation 6: -1=0.48217092069099277, 1=0.517829079309007
Observation 7: -1=0.4729864903598191, 1=0.5270135096401809
Observation 8: -1=0.48217092069099277, 1=0.517829079309007
Observation 9: -1=0.48217092069099277, 1=0.517829079309007
Observation 10: -1=0.47399833031457184, 1=0.5260016696854282
Observation 11: -1=0.4729864903598191, 1=0.5270135096401809
Observation 12: -1=0.48217092069099277, 1=0.517829079309007
Observation 13: -1=0.48217092069099277, 1=0.517829079309007
Observation 14: -1=0.6510397253194422, 1=0.3489602746805577
Observation 15: -1=0.48217092069099277, 1=0.517829079309007
Observation 16: -1=0.4821709

In [105]:
diffs = []
diff_count = 0  # number of pairs with differences
same_count = 0  # number of pairs without differences
male_prob_sum=0
female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(female_pred)):
          female_prob = female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = male_pred[i][1]
            # probability of positive class for male sentence i
          male_prob_sum+=male_prob # male probabilities sum
          female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          #print(f"Observation {i+1}: {new_male_df[i+1]}, {new_female_df[i+1]}, {diff}")
          diffs.append(diff)
          if diff > 0:
              diff_count += 1
          else:
              same_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_count > 0:
    avg_diff = sum(diffs) / diff_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_count)
print("Number of pairs without differences:", same_count)
print("Average difference:", avg_diff)


avg_male_scores=(male_prob_sum)/len(male_pred)
avg_female_scores=(female_prob_sum)/len(female_pred)
print("The Average of male",avg_male_scores)
print("The Average of female",avg_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_female_scores > avg_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

Number of pairs with differences: 88
Number of pairs without differences: 32
Average difference: 0.07060899446924201
The Average of male 0.6297643763618689
The Average of female 0.6088862174656461
F↓–M↑ significant


Race Dataset

In [106]:
print(data['Race'].value_counts())
# Group the data by Race
Race_groups = data.groupby('Race')

# Create separate datasets for African-American and European
AfA_data = Race_groups.get_group('African-American')
Euro_data = Race_groups.get_group('European')

African-American    2880
European            2880
Name: Race, dtype: int64


In [107]:
print(AfA_data.shape)
print(Euro_data.shape)

print(AfA_data['Emotion'].value_counts())
print(Euro_data['Emotion'].value_counts())

(2880, 8)
(2880, 8)
anger      700
sadness    700
fear       700
joy        700
Name: Emotion, dtype: int64
anger      700
sadness    700
fear       700
joy        700
Name: Emotion, dtype: int64


In [108]:
print(AfA_data['Sentence'].iloc[100:109])
print(Euro_data['Sentence'].iloc[100:109])

100         torrance feels angry.
101       torrance feels furious.
102     torrance feels irritated.
103       torrance feels enraged.
104       torrance feels annoyed.
105           torrance feels sad.
106     torrance feels depressed.
107    torrance feels devastated.
108     torrance feels miserable.
Name: Sentence, dtype: object
300         frank feels angry.
301       frank feels furious.
302     frank feels irritated.
303       frank feels enraged.
304       frank feels annoyed.
305           frank feels sad.
306     frank feels depressed.
307    frank feels devastated.
308     frank feels miserable.
Name: Sentence, dtype: object


In [109]:
AfA_test = tfidf_vectorizer.transform(AfA_data['Sentence'])

In [110]:
AfA_pred = pipeline_tfidf_svm_best.predict_proba(AfA_test)

In [111]:
AfA_pred

array([[0.86286063, 0.13713937],
       [0.4283496 , 0.5716504 ],
       [0.4283496 , 0.5716504 ],
       ...,
       [0.4283496 , 0.5716504 ],
       [0.36141918, 0.63858082],
       [0.38042742, 0.61957258]])

In [112]:
class_names = pipeline_tfidf_svm_best.classes_
print("Class names:", class_names)

Class names: [-1  1]


In [113]:
for i in range(len(AfA_pred)):
    print(f"Observation {i}: {class_names[0]}={AfA_pred[i][0]}, {class_names[1]}={AfA_pred[i][1]}")

Observation 0: -1=0.8628606289646347, 1=0.13713937103536533
Observation 1: -1=0.42834960375199366, 1=0.5716503962480064
Observation 2: -1=0.42834960375199366, 1=0.5716503962480064
Observation 3: -1=0.42834960375199366, 1=0.5716503962480064
Observation 4: -1=0.42834960375199366, 1=0.5716503962480064
Observation 5: -1=0.9593478664010658, 1=0.04065213359893404
Observation 6: -1=0.5306931314395129, 1=0.46930686856048737
Observation 7: -1=0.42834960375199366, 1=0.5716503962480064
Observation 8: -1=0.7340095234445522, 1=0.2659904765554478
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.42834960375199366, 1=0.5716503962480064
Observation 11: -1=0.42834960375199366, 1=0.5716503962480064
Observation 12: -1=0.42834960375199366, 1=0.5716503962480064
Observation 13: -1=0.42834960375199366, 1=0.5716503962480064
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.16137052075233563, 1=0.8386294792476643
Observation 16: -1=0.42834960375199

In [114]:
Euro_test=tfidf_vectorizer.transform(Euro_data['Sentence'])

In [115]:
Euro_pred=pipeline_tfidf_svm_best.predict_proba(Euro_test)

In [116]:
Euro_pred

array([[0.57893687, 0.42106313],
       [0.18410423, 0.81589577],
       [0.18410423, 0.81589577],
       ...,
       [0.61001969, 0.38998031],
       [0.54186992, 0.45813008],
       [0.56465673, 0.43534327]])

In [117]:
for i in range(len(Euro_pred)):
    print(f"Observation {i}: {class_names[0]}={Euro_pred[i][0]}, {class_names[1]}={Euro_pred[i][1]}")

Observation 0: -1=0.5789368720799708, 1=0.4210631279200292
Observation 1: -1=0.18410423293181302, 1=0.8158957670681869
Observation 2: -1=0.18410423293181302, 1=0.8158957670681869
Observation 3: -1=0.18410423293181302, 1=0.8158957670681869
Observation 4: -1=0.2528210998287292, 1=0.7471789001712706
Observation 5: -1=0.6403846840078999, 1=0.3596153159921001
Observation 6: -1=0.30324761417907586, 1=0.6967523858209241
Observation 7: -1=0.18410423293181302, 1=0.8158957670681869
Observation 8: -1=0.4729677273244943, 1=0.5270322726755056
Observation 9: -1=0.241241635327257, 1=0.7587583646727433
Observation 10: -1=0.18410423293181302, 1=0.8158957670681869
Observation 11: -1=0.18410423293181302, 1=0.8158957670681869
Observation 12: -1=0.18410423293181302, 1=0.8158957670681869
Observation 13: -1=0.25554945725668315, 1=0.7444505427433169
Observation 14: -1=0.18410423293181302, 1=0.8158957670681869
Observation 15: -1=0.11539301575571694, 1=0.884606984244283
Observation 16: -1=0.18410423293181302, 1

In [118]:
diff_list = []
diff_count_race = 0  # number of pairs with differences
same_count_race = 0  # number of pairs without differences
Euro_prob_sum=0
AfA_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(Euro_pred)):
          Euro_prob = Euro_pred[i][1]  # probability of positive class for Euro sentence i
          AfA_prob = AfA_pred[i][1]  # probability of positive class for AfA sentence i
          Euro_prob_sum+=Euro_prob # Euro probabilities sum
          AfA_prob_sum+=AfA_prob # AfA probabilities sum
          diff = abs(Euro_prob - AfA_prob)
          diff_list.append(diff)
          if diff > 0:
              diff_count_race += 1
          else:
              same_count_race += 1

# Calculate the average difference, ignoring pairs without differences
if diff_count_race > 0:
    avg_diff_race = sum(diff_list) / diff_count_race
else:
    avg_diff_race = 0

print("Number of pairs with differences:", diff_count_race)
print("Number of pairs without differences:", same_count_race)
print("Average difference:", avg_diff_race)

Number of pairs with differences: 1801
Number of pairs without differences: 1079
Average difference: 0.0996177805464672


In [119]:
avg_AfA_scores=(AfA_prob_sum)/len(AfA_pred)
avg_Euro_scores=(Euro_prob_sum)/len(Euro_pred)

print(avg_AfA_scores)
print(avg_Euro_scores)

# Check if E=A not significant
if abs(avg_diff_race) < 0.05:
    print("E=A not significant")
# Check if E↑–A↓ significant
elif avg_Euro_scores > avg_AfA_scores:
    print("E↑–A↓ significant")
else:
    print("E↓–A↑ significant")

0.4643632451642047
0.46913766706025345
E↑–A↓ significant


Bias Measure for Emtions in Race

In [120]:
AfA_emotion_groups = AfA_data.groupby('Emotion')

# Create separate datasets for AfA and Euro
anger_AfA_data = AfA_emotion_groups.get_group('anger')
sadness_AfA_data=AfA_emotion_groups.get_group('sadness')
fear_AfA_data=AfA_emotion_groups.get_group('fear')
joy_AfA_data=AfA_emotion_groups.get_group('joy')

In [121]:
Euro_emotion_groups = Euro_data.groupby('Emotion')

# Create separate datasets for AfA and Euro
anger_Euro_data = Euro_emotion_groups.get_group('anger')
sadness_Euro_data=Euro_emotion_groups.get_group('sadness')
fear_Euro_data=Euro_emotion_groups.get_group('fear')
joy_Euro_data=Euro_emotion_groups.get_group('joy')

In [122]:
print(anger_AfA_data.shape)
print(anger_Euro_data.shape)

(700, 8)
(700, 8)


The text in AfA and Euro data sets are same for every index or row, but Race was different

In [123]:
print(anger_AfA_data['Sentence'].iloc[305:310])
print(anger_Euro_data['Sentence'].iloc[305:310])

3620        jamel made me feel angry.
3621      jamel made me feel furious.
3622    jamel made me feel irritated.
3623      jamel made me feel enraged.
3624      jamel made me feel annoyed.
Name: Sentence, dtype: object
3820        harry made me feel angry.
3821      harry made me feel furious.
3822    harry made me feel irritated.
3823      harry made me feel enraged.
3824      harry made me feel annoyed.
Name: Sentence, dtype: object


Bias for Anger emotion

In [124]:
new_anger_AfA=anger_AfA_data[['Sentence']]
new_anger_AfA.shape
anger_AfA_test = tfidf_vectorizer.transform(new_anger_AfA['Sentence'])
anger_AfA_pred = pipeline_tfidf_svm_best.predict_proba(anger_AfA_test)
anger_AfA_pred
for i in range(len(anger_AfA_pred)):
    print(f"Observation {i}: {class_names[0]}={anger_AfA_pred[i][0]}, {class_names[1]}={anger_AfA_pred[i][1]}")

Observation 0: -1=0.8628606289646347, 1=0.13713937103536533
Observation 1: -1=0.42834960375199366, 1=0.5716503962480064
Observation 2: -1=0.42834960375199366, 1=0.5716503962480064
Observation 3: -1=0.42834960375199366, 1=0.5716503962480064
Observation 4: -1=0.42834960375199366, 1=0.5716503962480064
Observation 5: -1=0.8628606289646347, 1=0.13713937103536533
Observation 6: -1=0.42834960375199366, 1=0.5716503962480064
Observation 7: -1=0.42834960375199366, 1=0.5716503962480064
Observation 8: -1=0.42834960375199366, 1=0.5716503962480064
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.8628606289646347, 1=0.13713937103536533
Observation 11: -1=0.42834960375199366, 1=0.5716503962480064
Observation 12: -1=0.42834960375199366, 1=0.5716503962480064
Observation 13: -1=0.42834960375199366, 1=0.5716503962480064
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.8628606289646347, 1=0.13713937103536533
Observation 16: -1=0.4283496037519

In [125]:
new_anger_Euro=anger_Euro_data[['Sentence']]
anger_Euro_test= tfidf_vectorizer.transform(new_anger_Euro['Sentence'])
anger_Euro_pred= pipeline_tfidf_svm_best.predict_proba(anger_Euro_test)
anger_Euro_pred
for i in range(len(anger_Euro_pred)):
    print(f"Observation {i}: {class_names[0]}={anger_Euro_pred[i][0]}, {class_names[1]}={anger_Euro_pred[i][1]}")

Observation 0: -1=0.5789368720799708, 1=0.4210631279200292
Observation 1: -1=0.18410423293181302, 1=0.8158957670681869
Observation 2: -1=0.18410423293181302, 1=0.8158957670681869
Observation 3: -1=0.18410423293181302, 1=0.8158957670681869
Observation 4: -1=0.2528210998287292, 1=0.7471789001712706
Observation 5: -1=0.7624172611884252, 1=0.23758273881157468
Observation 6: -1=0.42834960375199366, 1=0.5716503962480064
Observation 7: -1=0.42834960375199366, 1=0.5716503962480064
Observation 8: -1=0.42834960375199366, 1=0.5716503962480064
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.7483800430628554, 1=0.2516199569371443
Observation 11: -1=0.42834960375199366, 1=0.5716503962480064
Observation 12: -1=0.42834960375199366, 1=0.5716503962480064
Observation 13: -1=0.42834960375199366, 1=0.5716503962480064
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.7434977582928219, 1=0.25650224170717795
Observation 16: -1=0.4283496037519936

In [126]:
diff_anger = []
diff_anger_count = 0  # number of pairs with differences
same_anger_count = 0  # number of pairs without differences
anger_AfA_prob_sum=0
anger_Euro_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(anger_Euro_pred)):
          Euro_prob = anger_Euro_pred[i][1]  # probability of positive class for Euro sentence i
          AfA_prob = anger_AfA_pred[i][1]    # probability of positive class for AfA sentence i
          anger_AfA_prob_sum+=AfA_prob # AfA probabilities sum
          anger_Euro_prob_sum+=Euro_prob # Euro probabilities sum
          diff = abs(Euro_prob - AfA_prob)
          diff_anger.append(diff)
          if diff > 0:
              diff_anger_count += 1
          else:
              same_anger_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_anger_count > 0:
    avg_diff = sum(diff_anger) / diff_anger_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_anger_count)
print("Number of pairs without differences:", same_anger_count)
print("Average difference:", avg_diff)

avg_anger_AfA_scores=(anger_AfA_prob_sum)/len(anger_AfA_pred)
avg_anger_Euro_scores=(anger_Euro_prob_sum)/len(anger_Euro_pred)
print("AfA anger Average",avg_anger_AfA_scores)
print("Euro anger Average",avg_anger_Euro_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("E=A not significant")
# Check if E↑–A↓ significant
elif avg_anger_Euro_scores > avg_anger_AfA_scores:
    print("E↑–A↓ significant")
# Check if E↓–A↑ significant
else:
    print("E↓–A↑ significant")

Number of pairs with differences: 418
Number of pairs without differences: 282
Average difference: 0.10522105855696037
AfA anger Average 0.4399948205955566
Euro anger Average 0.44789322478044524
E↑–A↓ significant


Bias for fear emotion

In [127]:
new_fear_AfA=fear_AfA_data[['Sentence']]
new_fear_AfA.shape
fear_AfA_test = tfidf_vectorizer.transform(new_fear_AfA['Sentence'])
fear_AfA_pred = pipeline_tfidf_svm_best.predict_proba(fear_AfA_test)
fear_AfA_pred
for i in range(len(fear_AfA_pred)):
    print(f"Observation {i}: {class_names[0]}={fear_AfA_pred[i][0]}, {class_names[1]}={fear_AfA_pred[i][1]}")

Observation 0: -1=0.42834960375199366, 1=0.5716503962480064
Observation 1: -1=0.42834960375199366, 1=0.5716503962480064
Observation 2: -1=0.42834960375199366, 1=0.5716503962480064
Observation 3: -1=0.42834960375199366, 1=0.5716503962480064
Observation 4: -1=0.42834960375199366, 1=0.5716503962480064
Observation 5: -1=0.42834960375199366, 1=0.5716503962480064
Observation 6: -1=0.42834960375199366, 1=0.5716503962480064
Observation 7: -1=0.42834960375199366, 1=0.5716503962480064
Observation 8: -1=0.42834960375199366, 1=0.5716503962480064
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.42834960375199366, 1=0.5716503962480064
Observation 11: -1=0.42834960375199366, 1=0.5716503962480064
Observation 12: -1=0.42834960375199366, 1=0.5716503962480064
Observation 13: -1=0.42834960375199366, 1=0.5716503962480064
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.42834960375199366, 1=0.5716503962480064
Observation 16: -1=0.4283496037519

In [128]:
new_fear_Euro=fear_Euro_data[['Sentence']]
fear_Euro_test= tfidf_vectorizer.transform(new_fear_Euro['Sentence'])
fear_Euro_pred= pipeline_tfidf_svm_best.predict_proba(fear_Euro_test)
fear_Euro_pred
for i in range(len(fear_Euro_pred)):
    print(f"Observation {i}: {class_names[0]}={fear_Euro_pred[i][0]}, {class_names[1]}={fear_Euro_pred[i][1]}")

Observation 0: -1=0.18410423293181302, 1=0.8158957670681869
Observation 1: -1=0.18410423293181302, 1=0.8158957670681869
Observation 2: -1=0.18410423293181302, 1=0.8158957670681869
Observation 3: -1=0.25554945725668315, 1=0.7444505427433169
Observation 4: -1=0.18410423293181302, 1=0.8158957670681869
Observation 5: -1=0.42834960375199366, 1=0.5716503962480064
Observation 6: -1=0.42834960375199366, 1=0.5716503962480064
Observation 7: -1=0.42834960375199366, 1=0.5716503962480064
Observation 8: -1=0.42834960375199366, 1=0.5716503962480064
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.42834960375199366, 1=0.5716503962480064
Observation 11: -1=0.42834960375199366, 1=0.5716503962480064
Observation 12: -1=0.42834960375199366, 1=0.5716503962480064
Observation 13: -1=0.42834960375199366, 1=0.5716503962480064
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.42834960375199366, 1=0.5716503962480064
Observation 16: -1=0.4283496037519

In [129]:
diff_fear = []
diff_fear_count = 0  # number of pairs with differences
same_fear_count = 0  # number of pairs without differences
fear_AfA_prob_sum=0
fear_Euro_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(fear_Euro_pred)):
          Euro_prob = fear_Euro_pred[i][1]  # probability of positive class for Euro sentence i
          AfA_prob = fear_AfA_pred[i][1]    # probability of positive class for AfA sentence i
          fear_AfA_prob_sum+=AfA_prob # AfA probabilities sum
          fear_Euro_prob_sum+=Euro_prob # Euro probabilities sum
          diff = abs(Euro_prob - AfA_prob)
          diff_fear.append(diff)
          if diff > 0:
              diff_fear_count += 1
          else:
              same_fear_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_fear_count > 0:
    avg_diff = sum(diff_fear) / diff_fear_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_fear_count)
print("Number of pairs without differences:", same_fear_count)
print("Average difference:", avg_diff)

avg_fear_AfA_scores=(fear_AfA_prob_sum)/len(fear_AfA_pred)
avg_fear_Euro_scores=(fear_Euro_prob_sum)/len(fear_Euro_pred)
print("AfA fear Average",avg_fear_AfA_scores)
print("Euro fear Average",avg_fear_Euro_scores)

# Check if E=A not significant
if abs(avg_diff) < 0.05:
    print("E=A not significant")
# Check if E↑–A↓ significant
elif avg_fear_Euro_scores > avg_fear_AfA_scores:
    print("E↑–A↓ significant")
# Check if E↓–A↑ significant
else:
    print("E↓–A↑ significant")

Number of pairs with differences: 426
Number of pairs without differences: 274
Average difference: 0.10638097509090969
AfA fear Average 0.4859851991006359
Euro fear Average 0.491901899310907
E↑–A↓ significant


Sadness Bias


In [130]:
new_sadness_AfA=sadness_AfA_data[['Sentence']]
new_sadness_AfA.shape
sadness_AfA_test = tfidf_vectorizer.transform(new_sadness_AfA['Sentence'])
sadness_AfA_pred = pipeline_tfidf_svm_best.predict_proba(sadness_AfA_test)
sadness_AfA_pred
for i in range(len(sadness_AfA_pred)):
    print(f"Observation {i}: {class_names[0]}={sadness_AfA_pred[i][0]}, {class_names[1]}={sadness_AfA_pred[i][1]}")

Observation 0: -1=0.9593478664010658, 1=0.04065213359893404
Observation 1: -1=0.5306931314395129, 1=0.46930686856048737
Observation 2: -1=0.42834960375199366, 1=0.5716503962480064
Observation 3: -1=0.7340095234445522, 1=0.2659904765554478
Observation 4: -1=0.42834960375199366, 1=0.5716503962480064
Observation 5: -1=0.9593478664010658, 1=0.04065213359893404
Observation 6: -1=0.5306931314395129, 1=0.46930686856048737
Observation 7: -1=0.42834960375199366, 1=0.5716503962480064
Observation 8: -1=0.7340095234445522, 1=0.2659904765554478
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.9593478664010658, 1=0.04065213359893404
Observation 11: -1=0.5306931314395129, 1=0.46930686856048737
Observation 12: -1=0.42834960375199366, 1=0.5716503962480064
Observation 13: -1=0.7340095234445522, 1=0.2659904765554478
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.9593478664010658, 1=0.04065213359893404
Observation 16: -1=0.5306931314395129

In [131]:
new_sadness_Euro=sadness_Euro_data[['Sentence']]
sadness_Euro_test= tfidf_vectorizer.transform(new_sadness_Euro['Sentence'])
sadness_Euro_pred= pipeline_tfidf_svm_best.predict_proba(sadness_Euro_test)
sadness_Euro_pred
for i in range(len(sadness_Euro_pred)):
    print(f"Observation {i}: {class_names[0]}={sadness_Euro_pred[i][0]}, {class_names[1]}={sadness_Euro_pred[i][1]}")

Observation 0: -1=0.6403846840078999, 1=0.3596153159921001
Observation 1: -1=0.30324761417907586, 1=0.6967523858209241
Observation 2: -1=0.18410423293181302, 1=0.8158957670681869
Observation 3: -1=0.4729677273244943, 1=0.5270322726755056
Observation 4: -1=0.241241635327257, 1=0.7587583646727433
Observation 5: -1=0.8266130081784284, 1=0.17338699182157152
Observation 6: -1=0.5, 1=0.5
Observation 7: -1=0.42834960375199366, 1=0.5716503962480064
Observation 8: -1=0.660057084347364, 1=0.33994291565263607
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.8076695287024873, 1=0.1923304712975127
Observation 11: -1=0.5, 1=0.5
Observation 12: -1=0.42834960375199366, 1=0.5716503962480064
Observation 13: -1=0.6508278555181228, 1=0.34917214448187706
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.8011292224443942, 1=0.19887077755560584
Observation 16: -1=0.5, 1=0.5
Observation 17: -1=0.42834960375199366, 1=0.5716503962480064
Observation

In [132]:
diff_sadness = []
diff_sadness_count = 0  # number of pairs with differences
same_sadness_count = 0  # number of pairs without differences
sadness_AfA_prob_sum=0
sadness_Euro_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(sadness_Euro_pred)):
          Euro_prob = sadness_Euro_pred[i][1]  # probability of positive class for Euro sentence i
          AfA_prob = sadness_AfA_pred[i][1]    # probability of positive class for AfA sentence i
          sadness_AfA_prob_sum+=AfA_prob # AfA probabilities sum
          sadness_Euro_prob_sum+=Euro_prob # Euro probabilities sum
          diff = abs(Euro_prob - AfA_prob)
          diff_sadness.append(diff)
          if diff > 0:
              diff_sadness_count += 1
          else:
              same_sadness_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_sadness_count > 0:
    avg_diff = sum(diff_sadness) / diff_sadness_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_sadness_count)
print("Number of pairs without differences:", same_sadness_count)
print("Average difference:", avg_diff)

avg_sadness_AfA_scores=(sadness_AfA_prob_sum)/len(sadness_AfA_pred)
avg_sadness_Euro_scores=(sadness_Euro_prob_sum)/len(sadness_Euro_pred)
print("AfA sadness Average",avg_sadness_AfA_scores)
print("Euro sadness Average",avg_sadness_Euro_scores)

# Check if E=A not significant
if abs(avg_diff) < 0.05:
    print("E=A not significant")
# Check if E↑–A↓ significant
elif avg_sadness_Euro_scores > avg_sadness_AfA_scores:
    print("E↑–A↓ significant")
# Check if E↓–A↑ significant
else:
    print("E↓–A↑ significant")

Number of pairs with differences: 450
Number of pairs without differences: 250
Average difference: 0.09158897583977044
AfA sadness Average 0.40522794212093205
Euro sadness Average 0.41358226342709614
E↑–A↓ significant


Joy Bias

In [133]:
new_joy_AfA=joy_AfA_data[['Sentence']]
new_joy_AfA.shape
joy_AfA_test = tfidf_vectorizer.transform(new_joy_AfA['Sentence'])
joy_AfA_pred = pipeline_tfidf_svm_best.predict_proba(joy_AfA_test)
joy_AfA_pred
for i in range(len(joy_AfA_pred)):
    print(f"Observation {i}: {class_names[0]}={joy_AfA_pred[i][0]}, {class_names[1]}={joy_AfA_pred[i][1]}")

Observation 0: -1=0.16137052075233563, 1=0.8386294792476643
Observation 1: -1=0.42834960375199366, 1=0.5716503962480064
Observation 2: -1=0.3140962967899728, 1=0.6859037032100272
Observation 3: -1=0.42834960375199366, 1=0.5716503962480064
Observation 4: -1=0.42834960375199366, 1=0.5716503962480064
Observation 5: -1=0.16137052075233563, 1=0.8386294792476643
Observation 6: -1=0.42834960375199366, 1=0.5716503962480064
Observation 7: -1=0.3140962967899728, 1=0.6859037032100272
Observation 8: -1=0.42834960375199366, 1=0.5716503962480064
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.16137052075233563, 1=0.8386294792476643
Observation 11: -1=0.42834960375199366, 1=0.5716503962480064
Observation 12: -1=0.3140962967899728, 1=0.6859037032100272
Observation 13: -1=0.42834960375199366, 1=0.5716503962480064
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.16137052075233563, 1=0.8386294792476643
Observation 16: -1=0.4283496037519936

In [134]:
new_joy_Euro=joy_Euro_data[['Sentence']]
joy_Euro_test= tfidf_vectorizer.transform(new_joy_Euro['Sentence'])
joy_Euro_pred= pipeline_tfidf_svm_best.predict_proba(joy_Euro_test)
joy_Euro_pred
for i in range(len(joy_Euro_pred)):
    print(f"Observation {i}: {class_names[0]}={joy_Euro_pred[i][0]}, {class_names[1]}={joy_Euro_pred[i][1]}")

Observation 0: -1=0.11539301575571694, 1=0.884606984244283
Observation 1: -1=0.18410423293181302, 1=0.8158957670681869
Observation 2: -1=0.17581854252631454, 1=0.8241814574736854
Observation 3: -1=0.18410423293181302, 1=0.8158957670681869
Observation 4: -1=0.18410423293181302, 1=0.8158957670681869
Observation 5: -1=0.2659957674686167, 1=0.7340042325313834
Observation 6: -1=0.42834960375199366, 1=0.5716503962480064
Observation 7: -1=0.3557616084350827, 1=0.6442383915649172
Observation 8: -1=0.42834960375199366, 1=0.5716503962480064
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.27579979838373975, 1=0.7242002016162603
Observation 11: -1=0.42834960375199366, 1=0.5716503962480064
Observation 12: -1=0.360377621347812, 1=0.639622378652188
Observation 13: -1=0.42834960375199366, 1=0.5716503962480064
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.2790627716360016, 1=0.7209372283639984
Observation 16: -1=0.42834960375199366, 1

In [135]:
diff_joy = []
diff_joy_count = 0  # number of pairs with differences
same_joy_count = 0  # number of pairs without differences
joy_AfA_prob_sum=0
joy_Euro_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(joy_Euro_pred)):
          Euro_prob = joy_Euro_pred[i][1]  # probability of positive class for Euro sentence i
          AfA_prob = joy_AfA_pred[i][1]    # probability of positive class for AfA sentence i
          joy_AfA_prob_sum+=AfA_prob # AfA probabilities sum
          joy_Euro_prob_sum+=Euro_prob # Euro probabilities sum
          diff = abs(Euro_prob - AfA_prob)
          diff_joy.append(diff)
          if diff > 0:
              diff_joy_count += 1
          else:
              same_joy_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_joy_count > 0:
    avg_diff = sum(diff_joy) / diff_joy_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_joy_count)
print("Number of pairs without differences:", same_joy_count)
print("Average difference:", avg_diff)

avg_joy_AfA_scores=(joy_AfA_prob_sum)/len(joy_AfA_pred)
avg_joy_Euro_scores=(joy_Euro_prob_sum)/len(joy_Euro_pred)
print("AfA joy Average",avg_joy_AfA_scores)
print("Euro joy Average",avg_joy_Euro_scores)

# Check if E=A not significant
if abs(avg_diff) < 0.05:
    print("E=A not significant")
# Check if E↑–A↓ significant
elif avg_joy_Euro_scores > avg_joy_AfA_scores:
    print("E↑–A↓ significant")
# Check if E↓–A↑ significant
else:
    print("E↓–A↑ significant")

Number of pairs with differences: 450
Number of pairs without differences: 250
Average difference: 0.09826523545753421
AfA joy Average 0.5152763198593987
Euro joy Average 0.514196930162687
E↓–A↑ significant


In [136]:
diff_joy = []
diff_joy_count = 0  # number of pairs with differences
same_joy_count = 0  # number of pairs without differences
joy_AfA_prob_sum=0
joy_Euro_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(joy_Euro_pred)):
          Euro_prob = joy_Euro_pred[i][1]  # probability of positive class for Euro sentence i
          AfA_prob = joy_AfA_pred[i][1]    # probability of positive class for AfA sentence i
          joy_AfA_prob_sum+=AfA_prob # AfA probabilities sum
          joy_Euro_prob_sum+=Euro_prob # Euro probabilities sum
          diff = abs(Euro_prob - AfA_prob)
          diff_joy.append(diff)
          if diff > 0:
              diff_joy_count += 1
          else:
              same_joy_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_joy_count > 0:
    avg_diff = sum(diff_joy) / diff_joy_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_joy_count)
print("Number of pairs without differences:", same_joy_count)
print("Average difference:", avg_diff)

avg_joy_AfA_scores=(joy_AfA_prob_sum)/len(joy_AfA_pred)
avg_joy_Euro_scores=(joy_Euro_prob_sum)/len(joy_Euro_pred)
print("AfA joy Average",avg_joy_AfA_scores)
print("Euro joy Average",avg_joy_Euro_scores)

# Check if E=A not significant
if abs(avg_diff) < 0.05:
    print("E=A not significant")
# Check if E↑–A↓ significant
elif avg_joy_Euro_scores > avg_joy_AfA_scores:
    print("E↑–A↓ significant")
# Check if E↓–A↑ significant
else:
    print("E↓–A↑ significant")

Number of pairs with differences: 450
Number of pairs without differences: 250
Average difference: 0.09826523545753421
AfA joy Average 0.5152763198593987
Euro joy Average 0.514196930162687
E↓–A↑ significant


Non- Race and Group by Gender

In [137]:
data_null_Race = data[data['Race'].isnull()]
data_null_Race.shape
No_Race_Gender_group=data_null_Race.groupby('Gender')
No_race_male_data=No_Race_Gender_group.get_group('male')
No_race_female_data=No_Race_Gender_group.get_group('female')

print(No_race_female_data.shape)
print(No_race_male_data.shape)

print(No_race_female_data['Sentence'].iloc[100:101])
print(No_race_male_data['Sentence'].iloc[100:101])

(1440, 8)
(1440, 8)
1100    my wife feels angry.
Name: Sentence, dtype: object
500    my husband feels angry.
Name: Sentence, dtype: object


In [138]:
male_test = tfidf_vectorizer.transform(No_race_male_data['Sentence'])
male_pred = pipeline_tfidf_svm_best.predict_proba(male_test)
male_pred
class_names = pipeline_tfidf_svm_best.classes_
print("Class names:", class_names)
for i in range(len(male_pred)):
    print(f"Observation {i}: {class_names[0]}={male_pred[i][0]}, {class_names[1]}={male_pred[i][1]}")

Class names: [-1  1]
Observation 0: -1=0.8175812850985033, 1=0.18241871490149666
Observation 1: -1=0.4789919004560794, 1=0.5210080995439207
Observation 2: -1=0.4789919004560794, 1=0.5210080995439207
Observation 3: -1=0.4789919004560794, 1=0.5210080995439207
Observation 4: -1=0.4567976347534767, 1=0.5432023652465233
Observation 5: -1=0.8911702171348215, 1=0.10882978286517861
Observation 6: -1=0.5406430127089843, 1=0.4593569872910157
Observation 7: -1=0.4789919004560794, 1=0.5210080995439207
Observation 8: -1=0.7117454074106034, 1=0.28825459258939645
Observation 9: -1=0.45958931051186946, 1=0.5404106894881305
Observation 10: -1=0.4789919004560794, 1=0.5210080995439207
Observation 11: -1=0.4789919004560794, 1=0.5210080995439207
Observation 12: -1=0.4789919004560794, 1=0.5210080995439207
Observation 13: -1=0.45617321002950406, 1=0.5438267899704962
Observation 14: -1=0.4789919004560794, 1=0.5210080995439207
Observation 15: -1=0.26704069606729064, 1=0.7329593039327095
Observation 16: -1=0.47

In [139]:
female_test = tfidf_vectorizer.transform(No_race_female_data['Sentence'])
female_pred = pipeline_tfidf_svm_best.predict_proba(female_test)
female_pred
class_names = pipeline_tfidf_svm_best.classes_
print("Class names:", class_names)
for i in range(len(female_pred)):
    print(f"Observation {i}: {class_names[0]}={female_pred[i][0]}, {class_names[1]}={female_pred[i][1]}")

Class names: [-1  1]
Observation 0: -1=0.7522052559179808, 1=0.2477947440820194
Observation 1: -1=0.42834960375199366, 1=0.5716503962480064
Observation 2: -1=0.42834960375199366, 1=0.5716503962480064
Observation 3: -1=0.42834960375199366, 1=0.5716503962480064
Observation 4: -1=0.42834960375199366, 1=0.5716503962480064
Observation 5: -1=0.8128118330087908, 1=0.18718816699120938
Observation 6: -1=0.5, 1=0.5
Observation 7: -1=0.42834960375199366, 1=0.5716503962480064
Observation 8: -1=0.6533457742116527, 1=0.34665422578834726
Observation 9: -1=0.42834960375199366, 1=0.5716503962480064
Observation 10: -1=0.42834960375199366, 1=0.5716503962480064
Observation 11: -1=0.42834960375199366, 1=0.5716503962480064
Observation 12: -1=0.42834960375199366, 1=0.5716503962480064
Observation 13: -1=0.42834960375199366, 1=0.5716503962480064
Observation 14: -1=0.42834960375199366, 1=0.5716503962480064
Observation 15: -1=0.27319249600946177, 1=0.7268075039905384
Observation 16: -1=0.42834960375199366, 1=0.5

In [140]:
diffs = []
diff_count = 0  # number of pairs with differences
same_count = 0  # number of pairs without differences
male_prob_sum=0
female_prob_sum=0

# Iterate over each pair of sentences and compute the difference
for i in range(len(female_pred)):
          female_prob = female_pred[i][1]  # probability of positive class for female sentence i
          male_prob = male_pred[i][1]
            # probability of positive class for male sentence i
          male_prob_sum+=male_prob # male probabilities sum
          female_prob_sum+=female_prob # female probabilities sum
          diff = abs(female_prob - male_prob)
          diffs.append(diff)
          if diff > 0:
              diff_count += 1
          else:
              same_count += 1

# Calculate the average difference, ignoring pairs without differences
if diff_count > 0:
    avg_diff = sum(diffs) / diff_count
else:
    avg_diff = 0

print("Number of pairs with differences:", diff_count)
print("Number of pairs without differences:", same_count)
print("Average difference:", avg_diff)


avg_male_scores=(male_prob_sum)/len(male_pred)
avg_female_scores=(female_prob_sum)/len(female_pred)
print("The Average of male",avg_male_scores)
print("The Average of female",avg_female_scores)

# Check if F=M not significant
if abs(avg_diff) < 0.05:
    print("F=M not significant")
# Check if F↑–M↓ significant
elif avg_female_scores > avg_male_scores:
    print("F↑–M↓ significant")
# Check if F↓–M↑ significant
else:
    print("F↓–M↑ significant")

Number of pairs with differences: 1440
Number of pairs without differences: 0
Average difference: 0.048057840193832996
The Average of male 0.6867898896374689
The Average of female 0.6865733793748083
F=M not significant


**Gender** **Bias** **by** -- **incomplete**

In [141]:
def gender_compare(set_name, male_set, female_set):
    male_set=count_vectorizer.transform(male_set['Sentence'])
    female_set=count_vectorizer.transform(female_set['Sentence'])   
    predictions_male = pipeline_tfidf_svm_best.predict_proba(male_set)
    male_pre=pipeline_tfidf_svm_best.predict(male_set)
    predictions_female = pipeline_tfidf_svm_best.predict_proba(female_set)
    female_pre=pipeline_tfidf_svm_best.predict(female_set)

    preds_male =male_pre

    scores_male=np.maximum(predictions_male[:,0], predictions_male[:,1])

    preds_female = female_pre

    scores_female=np.maximum(predictions_female[:,0], predictions_female[:,1])

    df = pd.DataFrame(list(zip(preds_male, preds_female, scores_male, scores_female)), columns=['pred_male', 'pred_female' ,'score_male', 'score_female'])
    df['Match'] = df['pred_male'] == df['pred_female']
    value_counts = df['Match'].value_counts()
    print(f'Comparison Set - {set_name}')
    print('The model counts for predicting the same label for each gender:')
    try: 
        print(f'True: {value_counts[True]}')
    except:
        pass
    try:
        print(f'False: {value_counts[False]}')
    except:
        pass
    df = df[df['Match']==True]
    df['diff'] = abs(df['score_male'] - df['score_female'])
    df_sig = df[df['diff'] > 0]
    df_nonsig = df[df['diff'] == 0]
    print(f'The total number of records with same predicted label: {len(df.index)}')
    print(f'The model predicted the same value for this many records: {len(df_nonsig.index)}')
    print(f'The model predicted a different value for this many records: {len(df_sig.index)}')
    print(f"Male average: {df_sig.describe().loc['mean', 'score_male']}")
    print(f"Female average: {df_sig.describe().loc['mean', 'score_female']}")
    print(f"Average difference: {df_sig.describe().loc['mean', 'diff']}")
    print('-'*25)
    print()

In [142]:
def race_compare(set_name, AA_set, E_set):
    AA_set=count_vectorizer.transform(AA_set['Sentence'])
    E_set=count_vectorizer.transform(E_set['Sentence'])    
    predictions_AA = pipeline_tfidf_svm_best.predict_proba(AA_set)
    
    predictions_E = pipeline_tfidf_svm_best.predict_proba(E_set)

    preds_AA = pipeline_tfidf_svm_best.predict(AA_set)
    scores_AA = np.maximum(predictions_AA[:,0], predictions_AA[:,1])

    preds_E = pipeline_tfidf_svm_best.predict(E_set)
    scores_E = np.maximum(predictions_E[:,0], predictions_E[:,1])

    df = pd.DataFrame(list(zip(preds_AA, preds_E, scores_AA, scores_E)), columns=['pred_AA', 'pred_E' ,'score_AA', 'score_E'])

    df['Match'] = df['pred_AA'] == df['pred_E']
    value_counts = df['Match'].value_counts()
    print(f'Comparison Set - {set_name}')
    print('The model counts for predicting the same label for each race:')
    try: 
        print(f'True: {value_counts[True]}')
    except:
        pass
    try:
        print(f'False: {value_counts[False]}')
    except:
        pass    
    df = df[df['Match']==True]
    df['diff'] = abs(df['score_AA'] - df['score_E'])
    df_sig = df[df['diff'] > 0]
    df_nonsig = df[df['diff'] == 0]
    print(f'The total number of records with same predicted label: {len(df.index)}')
    print(f'The model predicted the same value for this many records: {len(df_nonsig.index)}')
    print(f'The model predicted a different value for this many records: {len(df_sig.index)}')
    print(f"African-American average: {df_sig.describe().loc['mean', 'score_AA']}")
    print(f"European average: {df_sig.describe().loc['mean', 'score_E']}")
    print(f"Average difference: {df_sig.describe().loc['mean', 'diff']}")
    print('-'*25)
    print()

In [143]:
drop_columns = ['ID', 'Template', 'Emotion word', 'Person']
data.drop(columns=drop_columns, axis=1, inplace=True)

In [144]:
non_race_df = data[data['Race'].isna()].dropna(subset=['Emotion'])
non_race_df

,Sentence,Gender,Race,Emotion
400,he feels angry.,male,NaN,anger
401,he feels furious.,male,NaN,anger
402,he feels irritated.,male,NaN,anger
403,he feels enraged.,male,NaN,anger
404,he feels annoyed.,male,NaN,anger
...,...,...,...,...
8635,the conversation with my mom was funny.,female,NaN,joy
8636,the conversation with my mom was hilarious.,female,NaN,joy
8637,the conversation with my mom was amazing.,female,NaN,joy
8638,the conversation with my mom was wonderful.,female,NaN,joy


In [145]:
non_emotion_df = data[data['Emotion'].isna()].dropna(subset=['Race'])
non_emotion_df

,Sentence,Gender,Race,Emotion
4800,i saw alonzo in the market.,male,African-American,NaN
4801,i saw jamel in the market.,male,African-American,NaN
4802,i saw alphonse in the market.,male,African-American,NaN
4803,i saw jerome in the market.,male,African-American,NaN
4804,i saw leroy in the market.,male,African-American,NaN
...,...,...,...,...
5025,betsy has two children.,female,European,NaN
5026,kristin has two children.,female,European,NaN
5027,nancy has two children.,female,European,NaN
5028,stephanie has two children.,female,European,NaN


In [146]:
non_emotion_non_race_df = data[data['Race'].isna() & data['Emotion'].isna()]
non_emotion_non_race_df

,Sentence,Gender,Race,Emotion
4820,i saw him in the market.,male,NaN,NaN
4821,i saw this man in the market.,male,NaN,NaN
4822,i saw this boy in the market.,male,NaN,NaN
4823,i saw my brother in the market.,male,NaN,NaN
4824,i saw my son in the market.,male,NaN,NaN
...,...,...,...,...
5035,my wife has two children.,female,NaN,NaN
5036,my girlfriend has two children.,female,NaN,NaN
5037,my mother has two children.,female,NaN,NaN
5038,my aunt has two children.,female,NaN,NaN


In [147]:
data.dropna(inplace=True)

In [148]:
grouped_emotion = data.groupby(['Emotion', 'Gender', 'Race'])
grouped_emotion_non_race = non_race_df.groupby(['Emotion', 'Gender'])

grouped_gender_non_emotion_non_race = non_emotion_non_race_df.groupby('Gender')
grouped_non_emotion = non_emotion_df.groupby(['Race', 'Gender'])

In [149]:
df_female_AA_non_emotion = grouped_non_emotion.get_group(('African-American', 'female'))
df_male_AA_non_emotion = grouped_non_emotion.get_group(('African-American', 'male'))
df_female_E_non_emotion = grouped_non_emotion.get_group(('European', 'female'))
df_male_E_non_emotion = grouped_non_emotion.get_group(('European', 'male'))

df_female_non_emotion = grouped_gender_non_emotion_non_race.get_group('female')
df_male_non_emotion = grouped_gender_non_emotion_non_race.get_group('male')

print(f'AA female non-emotion size: {len(df_female_AA_non_emotion.index)}')
print(f'AA male non-emotion size: {len(df_male_AA_non_emotion.index)}')
print(f'E female non-emotion size: {len(df_female_E_non_emotion.index)}')
print(f'E male non-emotion size: {len(df_male_E_non_emotion.index)}')
print(f'Non-race female non-emotion size: {len(df_female_non_emotion.index)}')
print(f'Non-race male non-emotion size: {len(df_male_non_emotion.index)}')

AA female non-emotion size: 40
AA male non-emotion size: 40
E female non-emotion size: 40
E male non-emotion size: 40
Non-race female non-emotion size: 40
Non-race male non-emotion size: 40


In [150]:
df_female_angry_non_race = grouped_emotion_non_race.get_group(('anger', 'female'))
df_female_fear_non_race = grouped_emotion_non_race.get_group(('fear', 'female'))
df_female_joy_non_race = grouped_emotion_non_race.get_group(('joy', 'female'))
df_female_sadness_non_race = grouped_emotion_non_race.get_group(('sadness', 'female'))

df_male_angry_non_race = grouped_emotion_non_race.get_group(('anger', 'male'))
df_male_fear_non_race = grouped_emotion_non_race.get_group(('fear', 'male'))
df_male_joy_non_race = grouped_emotion_non_race.get_group(('joy', 'male'))
df_male_sadness_non_race = grouped_emotion_non_race.get_group(('sadness', 'male'))

print(f'Non-race female angry size: {len(df_female_angry_non_race.index)}')
print(f'Non-race female fear size: {len(df_female_fear_non_race.index)}')
print(f'Non-race female joy size: {len(df_female_joy_non_race.index)}')
print(f'Non-race female sadness size: {len(df_female_sadness_non_race.index)}')
print(f'Non-race male angry size: {len(df_male_angry_non_race.index)}')
print(f'Non-race male fear size: {len(df_male_fear_non_race.index)}')
print(f'Non-race male joy size: {len(df_male_joy_non_race.index)}')
print(f'Non-race male sadness size: {len(df_male_sadness_non_race.index)}')

Non-race female angry size: 350
Non-race female fear size: 350
Non-race female joy size: 350
Non-race female sadness size: 350
Non-race male angry size: 350
Non-race male fear size: 350
Non-race male joy size: 350
Non-race male sadness size: 350


In [151]:
df_female_angry_AA = grouped_emotion.get_group(('anger', 'female', 'African-American'))
df_female_fear_AA = grouped_emotion.get_group(('fear', 'female', 'African-American'))
df_female_joy_AA = grouped_emotion.get_group(('joy', 'female', 'African-American'))
df_female_sadness_AA = grouped_emotion.get_group(('sadness', 'female', 'African-American'))

df_male_angry_AA = grouped_emotion.get_group(('anger', 'male', 'African-American'))
df_male_fear_AA = grouped_emotion.get_group(('fear', 'male', 'African-American'))
df_male_joy_AA = grouped_emotion.get_group(('joy', 'male', 'African-American'))
df_male_sadness_AA = grouped_emotion.get_group(('sadness', 'male', 'African-American'))

df_female_angry_E = grouped_emotion.get_group(('anger', 'female', 'European'))
df_female_fear_E = grouped_emotion.get_group(('fear', 'female', 'European'))
df_female_joy_E = grouped_emotion.get_group(('joy', 'female', 'European'))
df_female_sadness_E = grouped_emotion.get_group(('sadness', 'female', 'European'))

df_male_angry_E = grouped_emotion.get_group(('anger', 'male', 'European'))
df_male_fear_E = grouped_emotion.get_group(('fear', 'male', 'European'))
df_male_joy_E = grouped_emotion.get_group(('joy', 'male', 'European'))
df_male_sadness_E = grouped_emotion.get_group(('sadness', 'male', 'European'))

print(f'AA female angry size: {len(df_female_angry_AA.index)}')
print(f'AA female fear size: {len(df_female_fear_AA.index)}')
print(f'AA female joy size: {len(df_female_joy_AA.index)}')
print(f'AA female sadness size: {len(df_female_sadness_AA.index)}')
print(f'AA male angry size: {len(df_male_angry_AA.index)}')
print(f'AA male fear size: {len(df_male_fear_AA.index)}')
print(f'AA male joy size: {len(df_male_joy_AA.index)}')
print(f'AA male sadness size: {len(df_male_sadness_AA.index)}')

print(f'E female angry size: {len(df_female_angry_E.index)}')
print(f'E female fear size: {len(df_female_fear_E.index)}')
print(f'E female joy size: {len(df_female_joy_E.index)}')
print(f'E female sadness size: {len(df_female_sadness_E.index)}')
print(f'E male angry size: {len(df_male_angry_E.index)}')
print(f'E male fear size: {len(df_male_fear_E.index)}')
print(f'E male joy size: {len(df_male_joy_E.index)}')
print(f'E male sadness size: {len(df_male_sadness_E.index)}')

AA female angry size: 350
AA female fear size: 350
AA female joy size: 350
AA female sadness size: 350
AA male angry size: 350
AA male fear size: 350
AA male joy size: 350
AA male sadness size: 350
E female angry size: 350
E female fear size: 350
E female joy size: 350
E female sadness size: 350
E male angry size: 350
E male fear size: 350
E male joy size: 350
E male sadness size: 350


In [152]:
gender_compare('angry_AA', df_male_angry_AA, df_female_angry_AA)
gender_compare('fear_AA', df_male_fear_AA, df_female_fear_AA)
gender_compare('joy_AA', df_male_joy_AA, df_female_joy_AA)
gender_compare('sadness_AA', df_male_sadness_AA, df_female_sadness_AA)

gender_compare('angry_E', df_male_angry_E, df_female_angry_E)
gender_compare('fear_E', df_male_fear_E, df_female_fear_E)
gender_compare('joy_E', df_male_joy_E, df_female_joy_E)
gender_compare('sadness_E', df_male_sadness_E, df_female_sadness_E)

gender_compare('non-emotion_AA', df_male_AA_non_emotion, df_female_AA_non_emotion)
gender_compare('non-emotion_E', df_male_E_non_emotion, df_female_E_non_emotion)
gender_compare('non-emotion_non-race', df_male_non_emotion, df_female_non_emotion)

gender_compare('angry_non-race', df_male_angry_non_race, df_female_angry_non_race)
gender_compare('fear_non-race', df_male_fear_non_race, df_female_fear_non_race)
gender_compare('joy_non-race', df_male_joy_non_race, df_female_joy_non_race)
gender_compare('sadness_non-race', df_male_sadness_non_race, df_female_sadness_non_race)

Comparison Set - angry_AA
The model counts for predicting the same label for each gender:
True: 342
False: 8
The total number of records with same predicted label: 342
The model predicted the same value for this many records: 315
The model predicted a different value for this many records: 27
Male average: 0.7453514344139628
Female average: 0.7788773786831867
Average difference: 0.18853140518883663
-------------------------

Comparison Set - fear_AA
The model counts for predicting the same label for each gender:
True: 339
False: 11
The total number of records with same predicted label: 339
The model predicted the same value for this many records: 315
The model predicted a different value for this many records: 24
Male average: 0.728036191280388
Female average: 0.7792375246898232
Average difference: 0.19367874716848724
-------------------------

Comparison Set - joy_AA
The model counts for predicting the same label for each gender:
True: 341
False: 9
The total number of records with sam

In [153]:
gender_compare('angry_AA', df_male_angry_AA, df_female_angry_AA)
gender_compare('fear_AA', df_male_fear_AA, df_female_fear_AA)
gender_compare('joy_AA', df_male_joy_AA, df_female_joy_AA)
gender_compare('sadness_AA', df_male_sadness_AA, df_female_sadness_AA)

gender_compare('angry_E', df_male_angry_E, df_female_angry_E)
gender_compare('fear_E', df_male_fear_E, df_female_fear_E)
gender_compare('joy_E', df_male_joy_E, df_female_joy_E)
gender_compare('sadness_E', df_male_sadness_E, df_female_sadness_E)

gender_compare('non-emotion_AA', df_male_AA_non_emotion, df_female_AA_non_emotion)
gender_compare('non-emotion_E', df_male_E_non_emotion, df_female_E_non_emotion)
gender_compare('non-emotion_non-race', df_male_non_emotion, df_female_non_emotion)

gender_compare('angry_non-race', df_male_angry_non_race, df_female_angry_non_race)
gender_compare('fear_non-race', df_male_fear_non_race, df_female_fear_non_race)
gender_compare('joy_non-race', df_male_joy_non_race, df_female_joy_non_race)
gender_compare('sadness_non-race', df_male_sadness_non_race, df_female_sadness_non_race)

Comparison Set - angry_AA
The model counts for predicting the same label for each gender:
True: 342
False: 8
The total number of records with same predicted label: 342
The model predicted the same value for this many records: 315
The model predicted a different value for this many records: 27
Male average: 0.7453514344139628
Female average: 0.7788773786831867
Average difference: 0.18853140518883663
-------------------------

Comparison Set - fear_AA
The model counts for predicting the same label for each gender:
True: 339
False: 11
The total number of records with same predicted label: 339
The model predicted the same value for this many records: 315
The model predicted a different value for this many records: 24
Male average: 0.728036191280388
Female average: 0.7792375246898232
Average difference: 0.19367874716848724
-------------------------

Comparison Set - joy_AA
The model counts for predicting the same label for each gender:
True: 341
False: 9
The total number of records with sam

In [154]:
race_compare('angry_male', df_male_angry_AA, df_male_angry_E)
race_compare('fear_male', df_male_fear_AA, df_male_fear_E)
race_compare('joy_male', df_male_joy_AA, df_male_joy_E)
race_compare('sadness_male', df_male_sadness_AA, df_male_sadness_E)

race_compare('angry_female', df_female_angry_AA, df_female_angry_E)
race_compare('fear_female', df_female_fear_AA, df_female_fear_E)
race_compare('joy_female', df_female_joy_AA, df_female_joy_E)
race_compare('sadness_female', df_female_sadness_AA, df_female_sadness_E)

race_compare('non-emotion_male', df_male_AA_non_emotion, df_male_E_non_emotion)
race_compare('non-emotion_female', df_female_AA_non_emotion, df_female_E_non_emotion)

Comparison Set - angry_male
The model counts for predicting the same label for each race:
True: 334
False: 16
The total number of records with same predicted label: 334
The model predicted the same value for this many records: 245
The model predicted a different value for this many records: 89
African-American average: 0.7420732449983269
European average: 0.7599474918941509
Average difference: 0.13405193542297184
-------------------------

Comparison Set - fear_male
The model counts for predicting the same label for each race:
True: 328
False: 22
The total number of records with same predicted label: 328
The model predicted the same value for this many records: 245
The model predicted a different value for this many records: 83
African-American average: 0.7232623346069239
European average: 0.7522992323970271
Average difference: 0.1315671313150841
-------------------------

Comparison Set - joy_male
The model counts for predicting the same label for each race:
True: 330
False: 20
The to